# 아이디어 노트
- 컬럼중에 married와 husband인경우를 묶어보자 -> 1?
- workclass == ?, occupation ==?, woman -> 수입이 낮은집단 0으로 주기?
- 결혼 안한 여성? -> 0?

- Married-civ-spouse	Craft-repair	Husband	White	Male	0	0	40	United-States	117 
  이 조건이 굉장히 흔한데 이조건에 해당하는 사람은 수익이 높기도 낮기도 하고 가장 많이 분포해있다. 이 사람들을 어떻게 분류할 것이냐

# train에 있는 >50K의 모든 경우의 수 살펴보기
# 그 다음 <=50K의 모든 경우의 수 살펴보기

In [82]:
# 데이터 분석 라이브러리
import numpy as np
import pandas as pd

# 시각화 라이브러리
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *

# 모델링 라이브러리
from category_encoders.ordinal import OrdinalEncoder
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier

# 기타 라이브러리
import random
import gc
import os

from tqdm import trange
from tqdm import tqdm
from tqdm import tqdm_notebook, tnrange
import time

## 데이터불러오기

In [83]:
cd data/kakr-4th-competition

[Errno 2] No such file or directory: 'data/kakr-4th-competition'
/Users/pym/data/kakr-4th-competition


In [151]:
test = pd.read_csv("./test.csv")

train = pd.read_csv("./train.csv")

sample_submission = pd.read_csv("./sample_submission.csv")

## 데이터 분석시작

### train(수입이 높은 사람과 낮은 사람을 다 포함하여 경우의 수 계산)

In [152]:
train.head()

,id,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,0,40,Private,168538,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,60,United-States,>50K
1,1,17,Private,101626,9th,5,Never-married,Machine-op-inspct,Own-child,White,Male,0,0,20,United-States,<=50K
2,2,18,Private,353358,Some-college,10,Never-married,Other-service,Own-child,White,Male,0,0,16,United-States,<=50K
3,3,21,Private,151158,Some-college,10,Never-married,Prof-specialty,Own-child,White,Female,0,0,25,United-States,<=50K
4,4,24,Private,122234,Some-college,10,Never-married,Adm-clerical,Not-in-family,Black,Female,0,0,20,?,<=50K


In [153]:
train = train.drop('education_num', axis = 1)

train = train.drop('fnlwgt', axis = 1)

train = train.drop('id', axis = 1)

In [154]:
train['num'] = 1

In [155]:
all_case_of_train = train.groupby(train.columns.tolist(),as_index=False)[['num']].count()

print('경우의 수 :',len(all_case_of_train), '\n')

print('선택한 컬럼 :',all_case_of_train.columns.tolist(), '\n')

print('경우 중 숫자의 개수 unique :',sorted(all_case_of_train['num'].unique().tolist()), '\n')

경우의 수 : 23644 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 



In [156]:
all_case_of_train[all_case_of_train.num > 11]

,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,num
1106,20,?,Some-college,Never-married,?,Own-child,White,Female,0,0,40,United-States,<=50K,12
12632,39,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K,12
18914,51,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K,12


## 데이터 분리(수입이 높은사람과 낮은사람의 데이터프레임 구분)

In [180]:
data_up = train[train.income == '>50K'].drop('income', axis=1)
data_dn = train[train.income == '<=50K'].drop('income', axis=1)

In [181]:
data_up.reset_index(drop=True, inplace=True)
data_dn.reset_index(drop=True, inplace=True)

### 경우의수 보여주는 함수

In [203]:
def all_case(df, column_list):
    all_case_of_df = df.groupby(column_list,as_index=False)[['num']].count()
    print('경우의 수 :',len(all_case_of_df), '\n')

    print('선택한 컬럼 :',all_case_of_df.columns.tolist(), '\n')

    print('경우 중 숫자의 개수 unique :',sorted(all_case_of_df['num'].unique().tolist()), '\n')
    return pd.DataFrame(all_case_of_df).sort_values(by='num', ascending=False)

### 앞서 삭제한 컬럼을 제외하고 모두 groupby

#### 수입이 높은집단

In [204]:
all_case(data_up, data_up.columns.tolist())

경우의 수 : 5884 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
2847,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,8
2996,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,7
4185,50,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,6
2604,42,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,5
4017,49,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007,39,Private,Bachelors,Married-spouse-absent,Craft-repair,Not-in-family,White,Male,0,1564,50,?,1
2006,39,Private,Bachelors,Married-civ-spouse,Transport-moving,Husband,White,Male,0,1977,45,Philippines,1
2005,39,Private,Bachelors,Married-civ-spouse,Tech-support,Husband,White,Male,15024,0,40,United-States,1
2004,39,Private,Bachelors,Married-civ-spouse,Tech-support,Husband,White,Male,0,0,50,United-States,1


- 모든 경우의 수를 따져봤을때 5884개의 결과가 나왔다.

- 해당 결과중 가장 많은 수의 케이스는 43/Private/HS-grad/Married-civ-spouse/Craft-repair/Husband/White/Male/0/0/40/UnitedState

- 위의 케이스가 8명이 존재하며 가장 많았다.

#### 수입이 낮은 집단

In [210]:
all_case(data_dn, data_up.columns.tolist())

경우의 수 : 17760 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
1105,20,?,Some-college,Never-married,?,Own-child,White,Female,0,0,40,United-States,12
14549,51,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,12
10614,39,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,12
7990,33,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,11
2006,22,?,Some-college,Never-married,?,Own-child,White,Male,0,0,40,United-States,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6294,30,?,Bachelors,Married-civ-spouse,?,Wife,White,Female,0,0,4,United-States,1
6295,30,?,Bachelors,Married-civ-spouse,?,Wife,White,Female,0,0,20,United-States,1
6296,30,?,Bachelors,Never-married,?,Not-in-family,Asian-Pac-Islander,Female,0,0,50,Taiwan,1
6297,30,?,Bachelors,Never-married,?,Not-in-family,Asian-Pac-Islander,Female,0,0,99,United-States,1


## 컬럼을 한개씩 빼고 groupby

### age 한개만 삭제

In [209]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [211]:
all_case(data_up, [
 'workclass',
 'education',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'num']) #age 한개만 삭제

경우의 수 : 4008 

선택한 컬럼 : ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 23, 24, 25, 27, 30, 35, 37, 45, 50, 56, 69, 90] 



,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
1835,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,90
1222,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,69
1364,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,56
2620,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,50
1234,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,45
...,...,...,...,...,...,...,...,...,...,...,...,...
1444,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,White,Female,7688,0,32,United-States,1
1445,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,White,Female,7688,0,36,United-States,1
1446,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,White,Female,7688,0,40,United-States,1
1447,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,White,Female,7688,0,43,United-States,1


- 나이만 빼고 돌렸는데도 많은 변화가 생김

- 우선적으로 전체 경우의 수가 4008개로 감소

- 모든것을 그룹바이 해서 돌렸을때 와 같은 값이 제일 빈도수가 높다.

#### 수입이 낮은 집단

In [212]:
all_case(data_dn, [
 'workclass',
 'education',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'num']) #age 한개만 삭제

경우의 수 : 12196 

선택한 컬럼 : ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 35, 36, 37, 38, 42, 43, 44, 49, 51, 55, 73, 82, 146, 252] 



,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
6296,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,252
6474,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,146
8825,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,82
6773,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,73
7010,Private,HS-grad,Never-married,Craft-repair,Not-in-family,White,Male,0,0,40,United-States,55
...,...,...,...,...,...,...,...,...,...,...,...,...
4292,Private,Assoc-acdm,Never-married,Other-service,Unmarried,White,Female,0,0,35,United-States,1
4293,Private,Assoc-acdm,Never-married,Priv-house-serv,Not-in-family,White,Female,0,0,50,United-States,1
4294,Private,Assoc-acdm,Never-married,Prof-specialty,Not-in-family,Black,Female,0,0,25,United-States,1
4295,Private,Assoc-acdm,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,33,United-States,1


### workclass 한개만 삭제

In [213]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은집단

In [214]:
all_case(data_up, ['age',
 'education',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'num']) #workclass한개만 삭제

경우의 수 : 5608 

선택한 컬럼 : ['age', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 8] 



,age,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
2852,44,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,8
3490,47,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,8
2719,43,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,8
3823,49,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,8
1939,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,8
...,...,...,...,...,...,...,...,...,...,...,...,...
1945,39,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,1
1943,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,7298,0,50,United-States,1
1942,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,7298,0,40,United-States,1
1941,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,5178,0,50,United-States,1


#### 수입이 낮은 집단

In [215]:
all_case(data_dn, ['age',
 'education',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'num']) #workclass한개만 삭제

경우의 수 : 17208 

선택한 컬럼 : ['age', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16] 



,age,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
7741,33,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,16
5826,29,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,16
9494,37,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,15
10279,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,14
14065,51,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,14
...,...,...,...,...,...,...,...,...,...,...,...,...
6144,30,7th-8th,Married-civ-spouse,Craft-repair,Husband,White,Male,2829,0,40,United-States,1
6145,30,7th-8th,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,1
6146,30,7th-8th,Never-married,Craft-repair,Not-in-family,White,Male,0,0,50,United-States,1
6147,30,7th-8th,Never-married,Craft-repair,Own-child,White,Male,0,0,40,United-States,1


### education 한개만 삭제

In [217]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수익이 높은 집단

In [220]:
all_case(data_up, ['age', 'workclass',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'num']) #education한개만 삭제

경우의 수 : 5277 

선택한 컬럼 : ['age', 'workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 



,age,workclass,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
3705,50,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,11
2695,44,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,10
2515,43,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,10
1407,37,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,9
1983,40,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,9
...,...,...,...,...,...,...,...,...,...,...,...,...
1855,39,Private,Married-civ-spouse,Sales,Husband,White,Male,3103,0,48,United-States,1
1854,39,Private,Married-civ-spouse,Sales,Husband,White,Male,3103,0,44,United-States,1
1853,39,Private,Married-civ-spouse,Sales,Husband,White,Male,0,2415,99,United-States,1
1852,39,Private,Married-civ-spouse,Sales,Husband,White,Male,0,1887,46,United-States,1


#### 수익이 낮은 집단

In [221]:
all_case(data_dn, ['age', 'workclass',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'num']) #education한개만 삭제

경우의 수 : 15807 

선택한 컬럼 : ['age', 'workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20] 



,age,workclass,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
8453,37,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,20
9185,39,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,20
8088,36,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,18
6409,32,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,17
5526,30,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,17
...,...,...,...,...,...,...,...,...,...,...,...,...
5755,30,Private,Never-married,Sales,Own-child,White,Male,0,0,30,United-States,1
5757,30,Private,Never-married,Sales,Unmarried,Black,Female,0,0,25,United-States,1
5758,30,Private,Never-married,Sales,Unmarried,White,Female,0,0,35,Mexico,1
5759,30,Private,Never-married,Tech-support,Not-in-family,Asian-Pac-Islander,Male,0,0,40,United-States,1


### marital_status 한개만 삭제

In [222]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [227]:
all_case(data_up, ['age', 'workclass', 'education',
'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'num']) #marital_staus 한개만 삭제

경우의 수 : 5881 

선택한 컬럼 : ['age', 'workclass', 'education', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8] 



,age,workclass,education,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
2843,43,Private,HS-grad,Craft-repair,Husband,White,Male,0,0,40,United-States,8
2990,44,Private,Bachelors,Exec-managerial,Husband,White,Male,0,0,40,United-States,7
4181,50,Private,HS-grad,Craft-repair,Husband,White,Male,0,0,40,United-States,6
4014,49,Private,HS-grad,Craft-repair,Husband,White,Male,0,0,40,United-States,5
1000,34,Private,Bachelors,Exec-managerial,Husband,White,Male,0,0,50,United-States,5
...,...,...,...,...,...,...,...,...,...,...,...,...
2009,39,Private,Bachelors,Tech-support,Husband,White,Male,0,0,50,United-States,1
2008,39,Private,Bachelors,Sales,Husband,White,Male,7298,0,55,United-States,1
2007,39,Private,Bachelors,Sales,Husband,White,Male,3103,0,48,United-States,1
2006,39,Private,Bachelors,Sales,Husband,White,Male,0,1848,40,United-States,1


#### 수입이 낮은 집단

In [228]:
all_case(data_dn, ['age', 'workclass', 'education',
'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'num']) #marital_status한개만 삭제

경우의 수 : 17470 

선택한 컬럼 : ['age', 'workclass', 'education', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 



,age,workclass,education,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
10404,39,Private,HS-grad,Craft-repair,Husband,White,Male,0,0,40,United-States,12
1098,20,?,Some-college,?,Own-child,White,Female,0,0,40,United-States,12
14275,51,Private,HS-grad,Craft-repair,Husband,White,Male,0,0,40,United-States,12
1990,22,?,Some-college,?,Own-child,White,Male,0,0,40,United-States,11
5939,29,Private,HS-grad,Craft-repair,Husband,White,Male,0,0,40,United-States,11
...,...,...,...,...,...,...,...,...,...,...,...,...
6212,30,?,9th,?,Not-in-family,White,Female,0,0,30,United-States,1
6213,30,?,Assoc-voc,?,Unmarried,White,Female,0,0,40,United-States,1
6214,30,?,Bachelors,?,Not-in-family,Asian-Pac-Islander,Female,0,0,50,Taiwan,1
6215,30,?,Bachelors,?,Not-in-family,Asian-Pac-Islander,Female,0,0,99,United-States,1


### occupation 한개만 삭제

In [226]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [229]:
all_case(data_up, ['age', 'workclass', 'education', 'marital_status',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'num']) #occupation 한개만 삭제

경우의 수 : 5234 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15] 



,age,workclass,education,marital_status,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
3697,50,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,15
1786,39,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,13
2645,44,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,12
1968,40,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,11
1583,38,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,11
...,...,...,...,...,...,...,...,...,...,...,...,...
1836,39,Private,Some-college,Married-civ-spouse,Husband,Black,Male,0,0,52,United-States,1
1835,39,Private,Some-college,Married-civ-spouse,Husband,Asian-Pac-Islander,Male,0,0,84,United-States,1
1833,39,Private,Some-college,Divorced,Not-in-family,White,Female,8614,0,40,United-States,1
1832,39,Private,Some-college,Divorced,Not-in-family,White,Female,0,0,45,United-States,1


#### 수익이 낮은 집단

In [230]:
all_case(data_dn, ['age', 'workclass', 'education', 'marital_status',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'num']) #occupation 한개만 삭제

경우의 수 : 15148 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28] 



,age,workclass,education,marital_status,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
6503,33,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,28
1372,21,Private,HS-grad,Never-married,Own-child,White,Male,0,0,40,United-States,27
6080,32,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,26
8002,37,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,26
5245,30,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,25
...,...,...,...,...,...,...,...,...,...,...,...,...
5501,31,Local-gov,HS-grad,Divorced,Unmarried,Amer-Indian-Eskimo,Male,0,0,40,United-States,1
5502,31,Local-gov,HS-grad,Divorced,Unmarried,White,Male,0,0,40,United-States,1
5503,31,Local-gov,HS-grad,Married-civ-spouse,Husband,Black,Male,0,0,40,United-States,1
5504,31,Local-gov,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,1


### relationship 한개만 삭제

In [232]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수익이 높은 집단

In [235]:
all_case(data_up, ['age', 'workclass', 'education', 'marital_status', 'occupation',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'num']) #relationship 한개만 삭제

경우의 수 : 5881 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8] 



,age,workclass,education,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
2844,43,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,8
2993,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,White,Male,0,0,40,United-States,7
4182,50,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,6
3212,45,Private,Bachelors,Married-civ-spouse,Exec-managerial,White,Male,0,0,40,United-States,5
2015,39,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,5
...,...,...,...,...,...,...,...,...,...,...,...,...
2010,39,Private,Bachelors,Widowed,Prof-specialty,White,Female,0,0,50,Poland,1
2009,39,Private,Bachelors,Separated,Prof-specialty,Asian-Pac-Islander,Female,0,0,40,Philippines,1
2007,39,Private,Bachelors,Never-married,Prof-specialty,White,Male,0,0,40,?,1
2006,39,Private,Bachelors,Never-married,Exec-managerial,White,Male,0,0,50,United-States,1


#### 수입이 낮은 집단

In [236]:
all_case(data_dn, ['age', 'workclass', 'education', 'marital_status', 'occupation',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'num']) #relationship 한개만 삭제

경우의 수 : 17112 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] 



,age,workclass,education,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
1840,22,?,Some-college,Never-married,?,White,Male,0,0,40,United-States,13
1034,20,?,Some-college,Never-married,?,White,Female,0,0,40,United-States,12
10057,39,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,12
13915,51,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,12
2588,23,Private,HS-grad,Never-married,Craft-repair,White,Male,0,0,40,United-States,12
...,...,...,...,...,...,...,...,...,...,...,...,...
6156,30,Private,HS-grad,Never-married,Sales,Black,Female,0,0,25,United-States,1
6157,30,Private,HS-grad,Never-married,Sales,White,Female,0,0,35,Mexico,1
6158,30,Private,HS-grad,Never-married,Sales,White,Female,0,0,35,United-States,1
6159,30,Private,HS-grad,Never-married,Sales,White,Female,0,0,40,United-States,1


### race 한개만 삭제

In [238]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [239]:
all_case(data_up, ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'num']) #race 한개만 삭제

경우의 수 : 5840 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 8] 



,age,workclass,education,marital_status,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,native_country,num
2826,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,8
2971,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,40,United-States,8
4152,50,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,6
2586,42,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,40,United-States,5
1000,34,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,50,United-States,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1996,39,Private,Bachelors,Married-civ-spouse,Transport-moving,Husband,Male,0,1977,45,Philippines,1
1995,39,Private,Bachelors,Married-civ-spouse,Tech-support,Husband,Male,15024,0,40,United-States,1
1994,39,Private,Bachelors,Married-civ-spouse,Tech-support,Husband,Male,0,0,50,United-States,1
1993,39,Private,Bachelors,Married-civ-spouse,Sales,Husband,Male,7298,0,55,United-States,1


#### 수입이 낮은 집단

In [240]:
all_case(data_dn, ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'num']) #race 한개만 삭제

경우의 수 : 17390 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14] 



,age,workclass,education,marital_status,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,native_country,num
14203,51,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,14
5828,29,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,14
7752,33,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,13
10322,39,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,13
1061,20,?,Some-college,Never-married,?,Own-child,Female,0,0,40,United-States,13
...,...,...,...,...,...,...,...,...,...,...,...,...
6214,30,Private,Assoc-voc,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,England,1
6215,30,Private,Assoc-voc,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,48,United-States,1
6216,30,Private,Assoc-voc,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,55,United-States,1
6217,30,Private,Assoc-voc,Married-civ-spouse,Handlers-cleaners,Husband,Male,0,0,40,Mexico,1


### sex 한개만 삭제

In [242]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수익이 높은 집단

In [244]:
all_case(data_up, ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship','race',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'num']) #sex 한개만 삭제

경우의 수 : 5883 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8] 



,age,workclass,education,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country,num
2847,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,8
2996,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,0,0,40,United-States,7
4184,50,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,6
3661,47,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,5
1560,37,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,0,0,50,United-States,5
...,...,...,...,...,...,...,...,...,...,...,...,...
2007,39,Private,Bachelors,Married-spouse-absent,Craft-repair,Not-in-family,White,0,1564,50,?,1
2006,39,Private,Bachelors,Married-civ-spouse,Transport-moving,Husband,White,0,1977,45,Philippines,1
2005,39,Private,Bachelors,Married-civ-spouse,Tech-support,Husband,White,15024,0,40,United-States,1
2004,39,Private,Bachelors,Married-civ-spouse,Tech-support,Husband,White,0,0,50,United-States,1


#### 수익이 낮은 집단

In [246]:
all_case(data_dn, ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship','race',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'num']) #sex 한개만 삭제

경우의 수 : 17320 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16] 



,age,workclass,education,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country,num
1008,20,?,Some-college,Never-married,?,Own-child,White,0,0,40,United-States,16
598,19,?,Some-college,Never-married,?,Own-child,White,0,0,40,United-States,13
1406,21,?,Some-college,Never-married,?,Own-child,White,0,0,40,United-States,12
14112,51,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,12
1826,22,?,Some-college,Never-married,?,Own-child,White,0,0,40,United-States,12
...,...,...,...,...,...,...,...,...,...,...,...,...
6204,30,Private,HS-grad,Married-spouse-absent,Craft-repair,Other-relative,White,0,0,40,United-States,1
6205,30,Private,HS-grad,Married-spouse-absent,Sales,Unmarried,White,0,0,40,United-States,1
6206,30,Private,HS-grad,Never-married,Adm-clerical,Not-in-family,White,0,0,40,United-States,1
6207,30,Private,HS-grad,Never-married,Adm-clerical,Not-in-family,White,0,0,45,United-States,1


### capital_gain 한개만 삭제

In [247]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [248]:
all_case(data_up, ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship','race','sex',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'num']) #capital_gain 한개만 삭제

경우의 수 : 5662 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_loss,hours_per_week,native_country,num
2740,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,8
4016,50,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,8
2512,42,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,40,United-States,7
2880,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,40,United-States,7
2790,43,Private,Some-college,Married-civ-spouse,Sales,Husband,White,Male,0,40,United-States,6
...,...,...,...,...,...,...,...,...,...,...,...,...
1960,39,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,Male,1887,46,United-States,1
1958,39,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,Male,0,44,United-States,1
1957,39,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,Male,0,40,United-States,1
1956,39,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,50,United-States,1


#### 수입이 낮은 집단

In [249]:
all_case(data_dn, ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship','race','sex',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'num']) #capital_gain 한개만 삭제

경우의 수 : 17649 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_loss,hours_per_week,native_country,num
1097,20,?,Some-college,Never-married,?,Own-child,White,Female,0,40,United-States,12
10536,39,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,12
14448,51,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,12
7926,33,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,12
1991,22,?,Some-college,Never-married,?,Own-child,White,Male,0,40,United-States,11
...,...,...,...,...,...,...,...,...,...,...,...,...
6272,30,Local-gov,7th-8th,Married-civ-spouse,Transport-moving,Husband,White,Male,0,40,United-States,1
6273,30,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,0,45,United-States,1
6274,30,Local-gov,Assoc-acdm,Never-married,Protective-serv,Not-in-family,Black,Male,0,40,United-States,1
6275,30,Local-gov,Assoc-voc,Divorced,Protective-serv,Unmarried,Amer-Indian-Eskimo,Female,0,40,United-States,1


### capital_loss 한개만 삭제

In [251]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [252]:
all_case(data_up, ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship','race','sex','capital_gain',
 'hours_per_week',
 'native_country',
 'num']) #capital_loss 한개만 삭제

경우의 수 : 5774 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 9] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,hours_per_week,native_country,num
2785,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,9
2930,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,40,United-States,7
1528,37,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,50,United-States,6
4094,50,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,6
3582,47,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1993,39,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,Male,0,46,United-States,1
1992,39,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,Male,0,40,United-States,1
1991,39,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,50,United-States,1
1990,39,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,45,?,1


#### 수입이 낮은 집단

In [253]:
all_case(data_dn, ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship','race','sex','capital_gain',
 'hours_per_week',
 'native_country',
 'num']) #capital_loss 한개만 삭제

경우의 수 : 17659 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,hours_per_week,native_country,num
10533,39,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,12
14452,51,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,12
1092,20,?,Some-college,Never-married,?,Own-child,White,Female,0,40,United-States,12
9726,37,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,12
1988,22,?,Some-college,Never-married,?,Own-child,White,Male,0,40,United-States,11
...,...,...,...,...,...,...,...,...,...,...,...,...
6276,30,Local-gov,Assoc-acdm,Never-married,Protective-serv,Not-in-family,Black,Male,0,40,United-States,1
6277,30,Local-gov,Assoc-voc,Divorced,Protective-serv,Unmarried,Amer-Indian-Eskimo,Female,0,40,United-States,1
6278,30,Local-gov,Assoc-voc,Married-civ-spouse,Protective-serv,Husband,White,Male,0,40,United-States,1
6279,30,Local-gov,Assoc-voc,Never-married,Protective-serv,Not-in-family,White,Male,0,40,United-States,1


### hours_per_week 한개만 삭제

In [254]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [255]:
all_case(data_up, ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship','race','sex','capital_gain',
 'capital_loss',
 'native_country',
 'num']) #hours_per_week 한개만 삭제

경우의 수 : 5131 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,native_country,num
2755,45,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,United-States,11
1568,38,Private,Bachelors,Married-civ-spouse,Sales,Husband,White,Male,0,0,United-States,10
655,32,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,United-States,10
893,34,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,United-States,10
1365,37,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,United-States,10
...,...,...,...,...,...,...,...,...,...,...,...,...
1829,39,State-gov,Bachelors,Divorced,Protective-serv,Not-in-family,White,Male,0,0,Mexico,1
1828,39,Self-emp-not-inc,Some-college,Separated,Craft-repair,Not-in-family,White,Male,0,0,United-States,1
1827,39,Self-emp-not-inc,Some-college,Never-married,Prof-specialty,Not-in-family,White,Male,0,1564,United-States,1
1826,39,Self-emp-not-inc,Some-college,Married-civ-spouse,Sales,Husband,White,Male,7688,0,United-States,1


#### 수입이 낮은 집단

In [257]:
all_case(data_dn, ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship','race','sex','capital_gain',
 'capital_loss',
 'native_country',
 'num']) #hours_per_week 한개만 삭제

경우의 수 : 15118 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 25, 28, 33] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,native_country,num
660,20,?,Some-college,Never-married,?,Own-child,White,Female,0,0,United-States,33
572,19,Private,Some-college,Never-married,Other-service,Own-child,White,Female,0,0,United-States,28
587,19,Private,Some-college,Never-married,Sales,Own-child,White,Female,0,0,United-States,25
384,19,?,Some-college,Never-married,?,Own-child,White,Male,0,0,United-States,23
43,17,Private,10th,Never-married,Other-service,Own-child,White,Male,0,0,United-States,21
...,...,...,...,...,...,...,...,...,...,...,...,...
5598,31,Private,Some-college,Never-married,Machine-op-inspct,Not-in-family,White,Female,0,0,United-States,1
5599,31,Private,Some-college,Never-married,Other-service,Not-in-family,White,Female,0,0,United-States,1
5601,31,Private,Some-college,Never-married,Other-service,Own-child,White,Male,0,1721,United-States,1
5602,31,Private,Some-college,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,United-States,1


### native_country 한개만 삭제

In [258]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [259]:
all_case(data_up, ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship','race','sex','capital_gain',
'capital_loss','hours_per_week','num']) # native_country 한개만 삭제

경우의 수 : 5837 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,num
2823,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,8
3188,45,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,7
2972,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,7
4147,50,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,6
997,34,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,6
...,...,...,...,...,...,...,...,...,...,...,...,...
1997,39,Private,Doctorate,Married-civ-spouse,Prof-specialty,Husband,White,Male,99999,0,55,1
1996,39,Private,Bachelors,Widowed,Prof-specialty,Unmarried,White,Female,0,0,50,1
1995,39,Private,Bachelors,Separated,Prof-specialty,Unmarried,Asian-Pac-Islander,Female,0,0,40,1
1993,39,Private,Bachelors,Never-married,Exec-managerial,Not-in-family,White,Male,0,0,50,1


#### 수익이 낮은 집단

In [261]:
all_case(data_dn, ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship','race','sex','capital_gain',
'capital_loss','hours_per_week','num']) # native_country 한개만 삭제

경우의 수 : 17576 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,num
1086,20,?,Some-college,Never-married,?,Own-child,White,Female,0,0,40,13
10477,39,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,12
5928,29,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,12
14382,51,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,12
1965,22,?,Some-college,Never-married,?,Own-child,White,Male,0,0,40,11
...,...,...,...,...,...,...,...,...,...,...,...,...
6260,30,Private,10th,Divorced,Other-service,Not-in-family,Amer-Indian-Eskimo,Male,0,0,40,1
6262,30,Private,10th,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,55,1
6263,30,Private,10th,Never-married,Craft-repair,Not-in-family,White,Male,0,0,50,1
6264,30,Private,10th,Never-married,Machine-op-inspct,Other-relative,Asian-Pac-Islander,Male,0,0,40,1


## 컬럼을 두개씩 빼고 groupby

### age, workclass 빼고

In [264]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [266]:
all_case(data_up, ['education', 'marital_status', 'occupation', 'relationship','race','sex','capital_gain',
'capital_loss','hours_per_week','native_country','num']) #age, workclass 두개 삭제

경우의 수 : 3360 

선택한 컬럼 : ['education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 28, 30, 31, 33, 34, 36, 38, 39, 42, 43, 46, 54, 61, 81, 98, 117] 



,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
1656,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,117
740,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,98
943,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,81
754,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,61
2360,Masters,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,54
...,...,...,...,...,...,...,...,...,...,...,...
1254,Bachelors,Never-married,Other-service,Not-in-family,White,Male,8614,0,50,Italy,1
1255,Bachelors,Never-married,Prof-specialty,Not-in-family,Amer-Indian-Eskimo,Female,14344,0,40,United-States,1
1256,Bachelors,Never-married,Prof-specialty,Not-in-family,Asian-Pac-Islander,Male,13550,0,40,United-States,1
1257,Bachelors,Never-married,Prof-specialty,Not-in-family,Black,Female,14084,0,60,United-States,1


#### 수입이 낮은 집단

In [269]:
all_case(data_dn, ['education', 'marital_status', 'occupation', 'relationship','race','sex','capital_gain',
'capital_loss','hours_per_week','native_country','num']) #age, workclass 두개 삭제

경우의 수 : 10815 

선택한 컬럼 : ['education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 37, 38, 40, 44, 45, 46, 47, 48, 49, 53, 55, 57, 59, 61, 64, 66, 93, 112, 154, 307] 



,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
5610,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,307
5873,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,154
9047,Some-college,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,112
6245,HS-grad,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,93
4375,Bachelors,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,40,United-States,66
...,...,...,...,...,...,...,...,...,...,...,...
3804,Bachelors,Married-civ-spouse,Other-service,Husband,White,Male,0,0,35,United-States,1
3805,Bachelors,Married-civ-spouse,Other-service,Husband,White,Male,0,0,40,Cuba,1
3806,Bachelors,Married-civ-spouse,Other-service,Husband,White,Male,0,0,40,El-Salvador,1
3808,Bachelors,Married-civ-spouse,Other-service,Husband,White,Male,0,0,40,Yugoslavia,1


### age,  education 빼고

In [270]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [271]:
all_case(data_up, ['workclass', 'marital_status', 'occupation', 'relationship','race','sex','capital_gain',
'capital_loss','hours_per_week','native_country','num']) #age, education 두개 삭제

경우의 수 : 2983 

선택한 컬럼 : ['workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 36, 37, 43, 44, 47, 51, 57, 61, 67, 99, 108, 127, 163, 179] 



,workclass,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
895,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,179
1028,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,163
1418,Private,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,127
1043,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,108
1633,Private,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States,99
...,...,...,...,...,...,...,...,...,...,...,...
1087,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1977,48,United-States,1
1090,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1977,60,?,1
1092,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1977,62,United-States,1
1093,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1977,65,United-States,1


#### 수익이 낮은 집단

In [273]:
all_case(data_dn, ['workclass', 'marital_status', 'occupation', 'relationship','race','sex','capital_gain',
'capital_loss','hours_per_week','native_country','num']) #age, education 두개 삭제

경우의 수 : 8754 

선택한 컬럼 : ['workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 36, 37, 38, 39, 41, 43, 44, 45, 48, 49, 50, 51, 52, 53, 54, 55, 56, 60, 61, 62, 64, 68, 69, 77, 80, 86, 91, 93, 98, 99, 102, 104, 106, 125, 135, 139, 247, 475] 



,workclass,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
2824,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,475
3234,Private,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,247
4195,Private,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,139
3900,Private,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,135
3694,Private,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States,125
...,...,...,...,...,...,...,...,...,...,...,...
3296,Private,Married-civ-spouse,Machine-op-inspct,Own-child,White,Female,2105,0,40,United-States,1
3295,Private,Married-civ-spouse,Machine-op-inspct,Own-child,White,Female,0,0,30,United-States,1
3294,Private,Married-civ-spouse,Machine-op-inspct,Own-child,Asian-Pac-Islander,Female,0,0,40,Vietnam,1
3293,Private,Married-civ-spouse,Machine-op-inspct,Other-relative,White,Male,0,0,20,United-States,1


### age, marital_status 빼고

In [275]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [277]:
all_case(data_up, ['workclass', 'education', 'occupation', 'relationship','race','sex','capital_gain',
'capital_loss','hours_per_week','native_country','num']) #age, marital_staus 두개 삭제

경우의 수 : 3943 

선택한 컬럼 : ['workclass', 'education', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 23, 24, 25, 27, 30, 35, 37, 45, 50, 56, 69, 91] 



,workclass,education,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
1761,Private,HS-grad,Craft-repair,Husband,White,Male,0,0,40,United-States,91
1162,Private,Bachelors,Exec-managerial,Husband,White,Male,0,0,40,United-States,69
1363,Private,Bachelors,Prof-specialty,Husband,White,Male,0,0,40,United-States,56
2540,Private,Some-college,Craft-repair,Husband,White,Male,0,0,40,United-States,50
1174,Private,Bachelors,Exec-managerial,Husband,White,Male,0,0,50,United-States,45
...,...,...,...,...,...,...,...,...,...,...,...
1423,Private,Bachelors,Prof-specialty,Not-in-family,Asian-Pac-Islander,Male,13550,0,40,Cambodia,1
1424,Private,Bachelors,Prof-specialty,Not-in-family,Asian-Pac-Islander,Male,13550,0,40,United-States,1
1425,Private,Bachelors,Prof-specialty,Not-in-family,Black,Female,14084,0,60,United-States,1
1428,Private,Bachelors,Prof-specialty,Not-in-family,White,Female,0,0,45,United-States,1


#### 수입이 낮은 집단

In [278]:
all_case(data_dn, ['workclass', 'education', 'occupation', 'relationship','race','sex','capital_gain',
'capital_loss','hours_per_week','native_country','num']) #age, marital_staus 두개 삭제

경우의 수 : 11051 

선택한 컬럼 : ['workclass', 'education', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 29, 30, 31, 32, 33, 36, 37, 38, 39, 40, 42, 43, 47, 51, 54, 55, 59, 61, 73, 76, 82, 99, 106, 146, 252] 



,workclass,education,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
5562,Private,HS-grad,Craft-repair,Husband,White,Male,0,0,40,United-States,252
6162,Private,HS-grad,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,146
5341,Private,HS-grad,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,106
5648,Private,HS-grad,Craft-repair,Not-in-family,White,Male,0,0,40,United-States,99
7870,Private,Some-college,Craft-repair,Husband,White,Male,0,0,40,United-States,82
...,...,...,...,...,...,...,...,...,...,...,...
3959,Private,Assoc-acdm,Prof-specialty,Own-child,White,Male,0,0,40,France,1
3960,Private,Assoc-acdm,Prof-specialty,Own-child,White,Male,0,0,40,United-States,1
3961,Private,Assoc-acdm,Prof-specialty,Unmarried,Black,Female,0,0,35,United-States,1
3962,Private,Assoc-acdm,Prof-specialty,Unmarried,Black,Female,0,0,36,United-States,1


### age, occupation 빼고

In [279]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [280]:
all_case(data_up, ['workclass', 'education', 'marital_status', 'relationship','race','sex','capital_gain',
'capital_loss','hours_per_week','native_country','num']) #age, occupation 두개 삭제

경우의 수 : 3056 

선택한 컬럼 : ['workclass', 'education', 'marital_status', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 29, 30, 31, 37, 41, 44, 46, 50, 60, 77, 102, 157, 205, 243] 



,workclass,education,marital_status,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
1432,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,243
993,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,205
1999,Private,Some-college,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,157
1010,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,0,50,United-States,102
1001,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,0,45,United-States,77
...,...,...,...,...,...,...,...,...,...,...,...
1138,Private,Bachelors,Married-civ-spouse,Wife,Black,Female,0,0,35,Haiti,1
1140,Private,Bachelors,Married-civ-spouse,Wife,Black,Female,0,0,45,United-States,1
1141,Private,Bachelors,Married-civ-spouse,Wife,Black,Female,15024,0,40,United-States,1
1142,Private,Bachelors,Married-civ-spouse,Wife,White,Female,0,0,4,United-States,1


#### 수입이 낮은 집단

In [281]:
all_case(data_dn, ['workclass', 'education', 'marital_status', 'relationship','race','sex','capital_gain',
'capital_loss','hours_per_week','native_country','num']) #age, occupation 두개 삭제

경우의 수 : 8756 

선택한 컬럼 : ['workclass', 'education', 'marital_status', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39, 42, 44, 45, 48, 49, 56, 58, 61, 63, 65, 66, 67, 68, 72, 79, 82, 89, 90, 97, 104, 105, 106, 113, 114, 117, 127, 128, 131, 204, 231, 288, 708] 



,workclass,education,marital_status,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
4868,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,708
6427,Private,Some-college,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,288
5567,Private,HS-grad,Never-married,Own-child,White,Male,0,0,40,United-States,231
5295,Private,HS-grad,Never-married,Not-in-family,White,Male,0,0,40,United-States,204
3984,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,131
...,...,...,...,...,...,...,...,...,...,...,...
3191,Private,9th,Married-civ-spouse,Wife,Black,Female,0,0,10,United-States,1
3189,Private,9th,Married-civ-spouse,Other-relative,Asian-Pac-Islander,Male,0,0,40,Philippines,1
3188,Private,9th,Married-civ-spouse,Other-relative,Asian-Pac-Islander,Female,0,0,45,China,1
3187,Private,9th,Married-civ-spouse,Husband,White,Male,3781,0,50,Mexico,1


- 수입이 높은 집단과 낮은 집단을 비교했을때 Married-civ-spouse	Husband	White	Male	0	0	40	United-States 공통적으로 많았으나 학력이 달랐다. 수입이 높은 집단은 학력이 높았고, 수입이 낮은 집단은 학력이 상대적으로 낮았다.

### age, relationship 빼고

In [279]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [282]:
all_case(data_up, ['workclass', 'education', 'marital_status', 'occupation','race','sex','capital_gain',
'capital_loss','hours_per_week','native_country','num']) #age, relationship 두개 삭제

경우의 수 : 3983 

선택한 컬럼 : ['workclass', 'education', 'marital_status', 'occupation', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 23, 24, 25, 27, 28, 30, 35, 38, 45, 50, 56, 70, 90] 



,workclass,education,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
1822,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,90
1231,Private,Bachelors,Married-civ-spouse,Exec-managerial,White,Male,0,0,40,United-States,70
1388,Private,Bachelors,Married-civ-spouse,Prof-specialty,White,Male,0,0,40,United-States,56
2599,Private,Some-college,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,50
1243,Private,Bachelors,Married-civ-spouse,Exec-managerial,White,Male,0,0,50,United-States,45
...,...,...,...,...,...,...,...,...,...,...,...
1431,Private,Bachelors,Married-civ-spouse,Prof-specialty,White,Male,7688,0,40,Canada,1
1436,Private,Bachelors,Married-civ-spouse,Prof-specialty,White,Male,15024,0,45,United-States,1
1437,Private,Bachelors,Married-civ-spouse,Prof-specialty,White,Male,15024,0,50,Italy,1
1439,Private,Bachelors,Married-civ-spouse,Prof-specialty,White,Male,15024,0,70,United-States,1


#### 수입이 낮은 집단

In [283]:
all_case(data_dn, ['workclass', 'education', 'marital_status', 'occupation','race','sex','capital_gain',
'capital_loss','hours_per_week','native_country','num']) #age, relationship 두개 삭제

경우의 수 : 10862 

선택한 컬럼 : ['workclass', 'education', 'marital_status', 'occupation', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 44, 45, 48, 51, 52, 64, 72, 75, 82, 85, 94, 103, 122, 147, 253] 



,workclass,education,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
5678,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,253
5870,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,White,Male,0,0,40,United-States,147
6355,Private,HS-grad,Never-married,Craft-repair,White,Male,0,0,40,United-States,122
6261,Private,HS-grad,Never-married,Adm-clerical,White,Female,0,0,40,United-States,103
5246,Private,HS-grad,Divorced,Adm-clerical,White,Female,0,0,40,United-States,94
...,...,...,...,...,...,...,...,...,...,...,...
3895,Private,Assoc-acdm,Never-married,Craft-repair,White,Male,0,0,60,United-States,1
3896,Private,Assoc-acdm,Never-married,Craft-repair,White,Male,2354,0,40,United-States,1
3897,Private,Assoc-acdm,Never-married,Exec-managerial,Asian-Pac-Islander,Female,0,0,50,United-States,1
3898,Private,Assoc-acdm,Never-married,Exec-managerial,White,Female,0,0,50,United-States,1


### age, race 빼고

In [284]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [285]:
all_case(data_up, ['workclass', 'education', 'marital_status', 'occupation','relationship','sex','capital_gain',
'capital_loss','hours_per_week','native_country','num']) #age, race 두개 삭제

경우의 수 : 3864 

선택한 컬럼 : ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 21, 23, 24, 25, 26, 27, 28, 29, 32, 40, 41, 47, 54, 58, 71, 94] 



,workclass,education,marital_status,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,native_country,num
1759,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,94
1161,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,40,United-States,71
1302,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,Male,0,0,40,United-States,58
2512,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,54
1174,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,50,United-States,47
...,...,...,...,...,...,...,...,...,...,...,...
1415,Private,Bachelors,Married-civ-spouse,Sales,Husband,Male,0,1848,55,United-States,1
1416,Private,Bachelors,Married-civ-spouse,Sales,Husband,Male,0,1887,35,United-States,1
1418,Private,Bachelors,Married-civ-spouse,Sales,Husband,Male,0,1887,60,United-States,1
1421,Private,Bachelors,Married-civ-spouse,Sales,Husband,Male,0,1902,55,United-States,1


#### 수입이 낮은 집단

In [286]:
all_case(data_dn, ['workclass', 'education', 'marital_status', 'occupation','relationship','sex','capital_gain',
'capital_loss','hours_per_week','native_country','num']) #age, race 두개 삭제

경우의 수 : 11288 

선택한 컬럼 : ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 36, 37, 38, 39, 40, 43, 44, 47, 50, 51, 52, 55, 57, 62, 67, 88, 89, 165, 274] 



,workclass,education,marital_status,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,native_country,num
5855,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,274
6015,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,Male,0,0,40,United-States,165
8137,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,89
6282,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,Male,0,0,40,United-States,88
5981,Private,HS-grad,Married-civ-spouse,Handlers-cleaners,Husband,Male,0,0,40,United-States,67
...,...,...,...,...,...,...,...,...,...,...,...
4044,Private,Assoc-acdm,Never-married,Sales,Own-child,Female,0,0,65,United-States,1
4049,Private,Assoc-acdm,Never-married,Tech-support,Not-in-family,Male,0,0,30,United-States,1
4050,Private,Assoc-acdm,Never-married,Tech-support,Not-in-family,Male,0,0,45,United-States,1
4051,Private,Assoc-acdm,Never-married,Tech-support,Not-in-family,Male,0,0,56,United-States,1


### age, sex 빼고

In [284]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [289]:
all_case(data_up, ['workclass', 'education', 'marital_status', 'occupation','relationship','race','capital_gain',
'capital_loss','hours_per_week','native_country','num']) #age, sex 두개 삭제

경우의 수 : 3972 

선택한 컬럼 : ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 23, 24, 25, 27, 30, 35, 37, 45, 50, 56, 69, 90] 



,workclass,education,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country,num
1816,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,90
1214,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,0,0,40,United-States,69
1355,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,0,0,40,United-States,56
2587,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,50
1226,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,0,0,50,United-States,45
...,...,...,...,...,...,...,...,...,...,...,...
1432,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,White,0,1977,40,United-States,1
1433,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,White,7298,0,8,?,1
1434,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,White,7688,0,10,United-States,1
1435,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,White,7688,0,32,United-States,1


#### 수입이 낮은 집단

In [290]:
all_case(data_dn, ['workclass', 'education', 'marital_status', 'occupation','relationship','race','capital_gain',
'capital_loss','hours_per_week','native_country','num']) #age, sex 두개 삭제

경우의 수 : 11492 

선택한 컬럼 : ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 41, 43, 44, 47, 49, 50, 51, 52, 53, 56, 60, 61, 64, 68, 73, 82, 146, 252] 



,workclass,education,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country,num
5946,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,252
6124,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,0,0,40,United-States,146
8296,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,82
6422,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,White,0,0,40,United-States,73
812,?,Some-college,Never-married,?,Own-child,White,0,0,40,United-States,68
...,...,...,...,...,...,...,...,...,...,...,...
4087,Private,Assoc-acdm,Never-married,Other-service,Own-child,Black,0,0,40,United-States,1
4088,Private,Assoc-acdm,Never-married,Other-service,Own-child,Black,0,0,44,United-States,1
4089,Private,Assoc-acdm,Never-married,Other-service,Own-child,White,0,0,5,United-States,1
4090,Private,Assoc-acdm,Never-married,Other-service,Own-child,White,0,0,9,United-States,1


### age, capital_gain 빼고

In [288]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [291]:
all_case(data_up, ['workclass', 'education', 'marital_status', 'occupation','relationship','race','sex',
'capital_loss','hours_per_week','native_country','num']) #age, capital_gain 두개 삭제

경우의 수 : 3417 

선택한 컬럼 : ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 26, 27, 29, 30, 31, 32, 33, 35, 36, 39, 42, 45, 59, 65, 70, 92, 108] 



,workclass,education,marital_status,occupation,relationship,race,sex,capital_loss,hours_per_week,native_country,num
1582,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,108
1086,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,40,United-States,92
1195,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,40,United-States,70
1099,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,50,United-States,65
2223,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,59
...,...,...,...,...,...,...,...,...,...,...,...
1239,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,40,United-States,1
1240,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,4,United-States,1
1241,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,8,?,1
1242,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,10,United-States,1


#### 수입이 낮은 집단

In [292]:
all_case(data_dn, ['workclass', 'education', 'marital_status', 'occupation','relationship','race','sex',
'capital_loss','hours_per_week','native_country','num']) #age, capital_gain 두개 삭제

경우의 수 : 11768 

선택한 컬럼 : ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 36, 37, 38, 39, 43, 44, 46, 49, 52, 56, 76, 87, 155, 261] 



,workclass,education,marital_status,occupation,relationship,race,sex,capital_loss,hours_per_week,native_country,num
6120,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,261
6270,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,40,United-States,155
8499,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,87
6538,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,White,Male,0,40,United-States,76
6760,Private,HS-grad,Never-married,Craft-repair,Not-in-family,White,Male,0,40,United-States,56
...,...,...,...,...,...,...,...,...,...,...,...
4165,Private,Assoc-acdm,Never-married,Machine-op-inspct,Unmarried,White,Male,0,40,United-States,1
4166,Private,Assoc-acdm,Never-married,Machine-op-inspct,Unmarried,White,Male,0,45,United-States,1
4167,Private,Assoc-acdm,Never-married,Other-service,Not-in-family,Asian-Pac-Islander,Female,0,35,Philippines,1
4168,Private,Assoc-acdm,Never-married,Other-service,Not-in-family,Black,Female,0,40,United-States,1


### age, capital_loss 빼고

In [293]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [294]:
all_case(data_up, ['workclass', 'education', 'marital_status', 'occupation','relationship','race','sex',
'capital_gain','hours_per_week','native_country','num']) #age, capital_loss 두개 삭제

경우의 수 : 3685 

선택한 컬럼 : ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 25, 26, 27, 29, 30, 31, 35, 38, 46, 55, 57, 65, 73, 98] 



,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,hours_per_week,native_country,num
1692,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,98
1151,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,40,United-States,73
1270,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,40,United-States,65
1163,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,50,United-States,57
2396,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,55
...,...,...,...,...,...,...,...,...,...,...,...
1338,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,White,Female,15024,40,United-States,1
1339,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,White,Female,15024,50,United-States,1
1340,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,White,Female,99999,20,United-States,1
1341,Private,Bachelors,Married-civ-spouse,Protective-serv,Husband,Asian-Pac-Islander,Male,0,40,Philippines,1


#### 수입이 낮은 집단

In [295]:
all_case(data_dn, ['workclass', 'education', 'marital_status', 'occupation','relationship','race','sex',
'capital_gain','hours_per_week','native_country','num']) #age, capital_loss 두개 삭제

경우의 수 : 11864 

선택한 컬럼 : ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 38, 40, 42, 43, 44, 45, 49, 50, 52, 59, 75, 87, 147, 263] 



,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,hours_per_week,native_country,num
6157,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,263
6314,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,40,United-States,147
8577,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,87
6598,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,White,Male,0,40,United-States,75
6825,Private,HS-grad,Never-married,Craft-repair,Not-in-family,White,Male,0,40,United-States,59
...,...,...,...,...,...,...,...,...,...,...,...
4191,Private,Assoc-acdm,Never-married,Handlers-cleaners,Own-child,White,Male,0,32,United-States,1
4192,Private,Assoc-acdm,Never-married,Handlers-cleaners,Own-child,White,Male,0,40,United-States,1
4193,Private,Assoc-acdm,Never-married,Machine-op-inspct,Not-in-family,White,Male,0,48,United-States,1
4194,Private,Assoc-acdm,Never-married,Machine-op-inspct,Other-relative,White,Female,0,15,United-States,1


- 백인 남성은 흔하다. 어떻게 차별점을 줄것인가... 가중치 고민을 해보면... 좋을듯 백인-남성- 교육수준 or 백인-남성-결혼상태

### age, hours_per_week 빼고

In [296]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [297]:
all_case(data_up, ['workclass', 'education', 'marital_status', 'occupation','relationship','race','sex',
'capital_gain','capital_loss','native_country','num']) #age, hours_per_week 두개 삭제

경우의 수 : 2624 

선택한 컬럼 : ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 29, 30, 31, 32, 33, 34, 38, 58, 61, 68, 70, 85, 86, 112, 122, 148, 190] 



,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,native_country,num
898,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,United-States,190
1244,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,United-States,148
962,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,United-States,122
1006,Private,Bachelors,Married-civ-spouse,Sales,Husband,White,Male,0,0,United-States,112
1471,Private,Masters,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,United-States,86
...,...,...,...,...,...,...,...,...,...,...,...
958,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,Canada,1
956,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,Other,Male,0,0,?,1
955,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,Black,Male,0,1977,United-States,1
954,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,Black,Male,0,0,United-States,1


#### 수입이 낮은 집단

In [298]:
all_case(data_dn, ['workclass', 'education', 'marital_status', 'occupation','relationship','race','sex',
'capital_gain','capital_loss','native_country','num']) #age, hours_per_week 두개 삭제

경우의 수 : 7517 

선택한 컬럼 : ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 43, 45, 46, 47, 48, 49, 50, 51, 54, 55, 57, 58, 60, 61, 62, 64, 66, 69, 70, 73, 74, 75, 80, 83, 84, 85, 87, 93, 95, 96, 99, 101, 114, 130, 172, 198, 399] 



,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,native_country,num
4117,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,United-States,399
4205,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,United-States,198
4359,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,United-States,172
5492,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,United-States,130
4315,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,Male,0,0,United-States,114
...,...,...,...,...,...,...,...,...,...,...,...
2773,Private,9th,Separated,Handlers-cleaners,Own-child,White,Male,0,0,United-States,1
2772,Private,9th,Separated,Handlers-cleaners,Other-relative,White,Male,0,0,El-Salvador,1
2771,Private,9th,Separated,Farming-fishing,Unmarried,Black,Male,0,0,United-States,1
2770,Private,9th,Separated,Farming-fishing,Not-in-family,Black,Male,0,0,United-States,1


- 수입이 낮은 집단의 첫번째 row와 수입이 높은 집단의 두번째 row는 조건이 같으나 분류는 다르게 되었다. 현재 빠져있는 값은 주간 근무시간이다. 주간 근무시간을 토대로 비교해볼 필요가 있다.

### age, native_country 빼고

In [299]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [300]:
all_case(data_up, ['workclass', 'education', 'marital_status', 'occupation','relationship','race','sex',
'capital_gain','capital_loss','hours_per_week','num']) #age, native_country 두개 삭제

경우의 수 : 3825 

선택한 컬럼 : ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 22, 23, 24, 26, 27, 29, 31, 39, 40, 50, 52, 57, 74, 94] 



,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,num
1741,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,94
1169,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,74
1299,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,57
2484,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,52
1176,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,50
...,...,...,...,...,...,...,...,...,...,...,...
1388,Private,Bachelors,Married-civ-spouse,Sales,Husband,White,Male,0,0,26,1
1389,Private,Bachelors,Married-civ-spouse,Sales,Husband,White,Male,0,0,30,1
1399,Private,Bachelors,Married-civ-spouse,Sales,Husband,White,Male,0,1848,40,1
1400,Private,Bachelors,Married-civ-spouse,Sales,Husband,White,Male,0,1848,50,1


#### 수입이 낮은 집단

In [301]:
all_case(data_dn, ['workclass', 'education', 'marital_status', 'occupation','relationship','race','sex',
'capital_gain','capital_loss','hours_per_week','num']) #age, native_country 두개 삭제

경우의 수 : 11507 

선택한 컬럼 : ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 51, 53, 54, 58, 75, 91, 149, 265] 



,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,num
5972,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,265
6131,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,149
8289,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,91
6388,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,75
6603,Private,HS-grad,Never-married,Craft-repair,Not-in-family,White,Male,0,0,40,58
...,...,...,...,...,...,...,...,...,...,...,...
4071,Private,Assoc-acdm,Never-married,Exec-managerial,Unmarried,White,Female,2174,0,55,1
4072,Private,Assoc-acdm,Never-married,Exec-managerial,Unmarried,White,Male,0,0,50,1
4073,Private,Assoc-acdm,Never-married,Farming-fishing,Not-in-family,White,Male,2354,0,45,1
4074,Private,Assoc-acdm,Never-married,Handlers-cleaners,Not-in-family,White,Male,0,0,36,1


- 모든 컬럼에 대한 값이 동일한 조건이나 수입에 관해서만 다를경우에는 어떻게 분류할까? 비율? 해당 조건에서 수입이 높은 집단인 사람들의 비율로 하는 방법

### workclass, educaiton 빼고

In [302]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [303]:
all_case(data_up, ['age', 'marital_status', 'occupation','relationship','race','sex',
'capital_gain','capital_loss','hours_per_week', 'native_country','num']) #workclass, education 두개 삭제

경우의 수 : 4729 

선택한 컬럼 : ['age', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] 



,age,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
3185,49,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,13
2255,43,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,12
2476,44,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,11
2312,43,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,11
1986,41,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,11
...,...,...,...,...,...,...,...,...,...,...,...
1708,39,Married-civ-spouse,Sales,Husband,White,Male,3103,0,44,United-States,1
1709,39,Married-civ-spouse,Sales,Husband,White,Male,3103,0,48,United-States,1
1710,39,Married-civ-spouse,Sales,Husband,White,Male,3103,0,50,United-States,1
1711,39,Married-civ-spouse,Sales,Husband,White,Male,5178,0,48,United-States,1


#### 수입이 낮은 집단

In [304]:
all_case(data_dn, ['age', 'marital_status', 'occupation','relationship','race','sex',
'capital_gain','capital_loss','hours_per_week', 'native_country','num']) #workclass, education 두개 삭제

경우의 수 : 14748 

선택한 컬럼 : ['age', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 25, 27] 



,age,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
7919,37,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,27
8589,39,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,25
6391,33,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,23
6009,32,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,23
7178,35,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,22
...,...,...,...,...,...,...,...,...,...,...,...
5444,30,Never-married,Protective-serv,Not-in-family,Black,Male,0,0,40,United-States,1
5445,30,Never-married,Protective-serv,Not-in-family,Black,Male,0,1974,40,United-States,1
5446,30,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,1
5447,30,Never-married,Protective-serv,Own-child,Black,Female,0,0,40,United-States,1


### workclass, marital_status 빼고

In [305]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [306]:
all_case(data_up, ['age', 'education', 'occupation','relationship','race','sex',
'capital_gain','capital_loss','hours_per_week', 'native_country','num']) #workclass, marital_status 두개 삭제

경우의 수 : 5602 

선택한 컬럼 : ['age', 'education', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 8] 



,age,education,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
1932,39,HS-grad,Craft-repair,Husband,White,Male,0,0,40,United-States,8
2713,43,HS-grad,Craft-repair,Husband,White,Male,0,0,40,United-States,8
3483,47,HS-grad,Craft-repair,Husband,White,Male,0,0,40,United-States,8
3816,49,HS-grad,Craft-repair,Husband,White,Male,0,0,40,United-States,8
2843,44,Bachelors,Exec-managerial,Husband,White,Male,0,0,40,United-States,8
...,...,...,...,...,...,...,...,...,...,...,...
1945,39,HS-grad,Farming-fishing,Not-in-family,White,Male,8614,0,50,United-States,1
1944,39,HS-grad,Exec-managerial,Husband,White,Male,7298,0,40,United-States,1
1943,39,HS-grad,Exec-managerial,Husband,White,Male,3103,0,60,United-States,1
1942,39,HS-grad,Exec-managerial,Husband,White,Male,0,1977,40,United-States,1


#### 수입이 낮은 집단

In [307]:
all_case(data_dn, ['age', 'education', 'occupation','relationship','race','sex',
'capital_gain','capital_loss','hours_per_week', 'native_country','num']) #workclass, marital_status 두개 삭제

경우의 수 : 16814 

선택한 컬럼 : ['age', 'education', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16] 



,age,education,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
5731,29,HS-grad,Craft-repair,Husband,White,Male,0,0,40,United-States,16
7572,33,HS-grad,Craft-repair,Husband,White,Male,0,0,40,United-States,16
9245,37,HS-grad,Craft-repair,Husband,White,Male,0,0,40,United-States,15
13693,51,HS-grad,Craft-repair,Husband,White,Male,0,0,40,United-States,14
10013,39,HS-grad,Craft-repair,Husband,White,Male,0,0,40,United-States,14
...,...,...,...,...,...,...,...,...,...,...,...
6029,30,11th,Handlers-cleaners,Unmarried,White,Female,0,0,40,United-States,1
6030,30,11th,Other-service,Husband,White,Male,0,0,40,Mexico,1
6031,30,11th,Other-service,Own-child,White,Female,0,0,19,United-States,1
6032,30,11th,Other-service,Wife,Black,Female,0,0,30,United-States,1


- HS-grad	Craft-repair	Husband	White	Male 이조건 세분화 필요성 느낀다. 나이로 세분화 가능할듯(가정해보자면 경력이 영향에 미쳐서?)

### workclass, occupation 빼고

In [310]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [311]:
all_case(data_up, ['age', 'education', 'marital_status','relationship','race','sex',
'capital_gain','capital_loss','hours_per_week', 'native_country','num']) #workclass, occupation 두개 삭제

경우의 수 : 4656 

선택한 컬럼 : ['age', 'education', 'marital_status', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18] 



,age,education,marital_status,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
3272,50,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,18
3145,49,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,17
1605,39,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,17
3406,51,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,16
2353,44,Bachelors,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,15
...,...,...,...,...,...,...,...,...,...,...,...
1675,39,Some-college,Married-civ-spouse,Husband,Asian-Pac-Islander,Male,0,0,84,United-States,1
1676,39,Some-college,Married-civ-spouse,Husband,Black,Male,0,0,52,United-States,1
1677,39,Some-college,Married-civ-spouse,Husband,White,Male,0,0,40,Philippines,1
1679,39,Some-college,Married-civ-spouse,Husband,White,Male,0,0,45,United-States,1


#### 수입이 낮은 집단

In [312]:
all_case(data_dn, ['age', 'education', 'marital_status','relationship','race','sex',
'capital_gain','capital_loss','hours_per_week', 'native_country','num']) #workclass, occupation 두개 삭제

경우의 수 : 13640 

선택한 컬럼 : ['age', 'education', 'marital_status', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 36, 37] 



,age,education,marital_status,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
5436,32,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,37
5802,33,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,37
7152,37,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,36
1277,21,Some-college,Never-married,Own-child,White,Male,0,0,40,United-States,32
10877,51,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,31
...,...,...,...,...,...,...,...,...,...,...,...
5070,31,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,52,Mexico,1
5071,31,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,52,United-States,1
5075,31,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,70,United-States,1
5076,31,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,91,United-States,1


- 나이로 세분화 가능해보임.. 위아래 비교해보면 수입이 높은 집단이 나이가 높다

### workclass, relationship 빼고

In [313]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [314]:
all_case(data_up, ['age', 'education', 'marital_status','occupation','race','sex',
'capital_gain','capital_loss','hours_per_week', 'native_country','num']) #workclass, relationship 두개 삭제

경우의 수 : 5605 

선택한 컬럼 : ['age', 'education', 'marital_status', 'occupation', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 8] 



,age,education,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
3487,47,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,8
3821,49,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,8
2716,43,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,8
1936,39,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,8
2849,44,Bachelors,Married-civ-spouse,Exec-managerial,White,Male,0,0,40,United-States,8
...,...,...,...,...,...,...,...,...,...,...,...
1946,39,HS-grad,Married-civ-spouse,Exec-managerial,White,Male,7298,0,40,United-States,1
1945,39,HS-grad,Married-civ-spouse,Exec-managerial,White,Male,3103,0,60,United-States,1
1944,39,HS-grad,Married-civ-spouse,Exec-managerial,White,Male,0,1977,40,United-States,1
1942,39,HS-grad,Married-civ-spouse,Exec-managerial,White,Male,0,0,50,United-States,1


#### 수입이 낮은 집단

In [315]:
all_case(data_dn, ['age', 'education', 'marital_status','occupation','race','sex',
'capital_gain','capital_loss','hours_per_week', 'native_country','num']) #workclass, relationship 두개 삭제

경우의 수 : 16430 

선택한 컬럼 : ['age', 'education', 'marital_status', 'occupation', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16] 



,age,education,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
5353,29,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,16
7181,33,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,16
8871,37,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,15
9629,39,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,14
13309,51,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,14
...,...,...,...,...,...,...,...,...,...,...,...
5988,30,Some-college,Married-civ-spouse,Craft-repair,White,Male,0,0,58,United-States,1
5989,30,Some-college,Married-civ-spouse,Craft-repair,White,Male,0,1740,40,United-States,1
5990,30,Some-college,Married-civ-spouse,Craft-repair,White,Male,0,2051,40,United-States,1
5991,30,Some-college,Married-civ-spouse,Exec-managerial,White,Female,0,0,25,United-States,1


### workclass, race 빼고

In [316]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [317]:
all_case(data_up, ['age', 'education', 'marital_status','occupation','relationship','sex',
'capital_gain','capital_loss','hours_per_week', 'native_country','num']) #workclass, race 두개 삭제

경우의 수 : 5535 

선택한 컬럼 : ['age', 'education', 'marital_status', 'occupation', 'relationship', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 10] 



,age,education,marital_status,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,native_country,num
2813,44,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,40,United-States,10
1921,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,8
2687,43,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,8
3441,47,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,8
3767,49,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,8
...,...,...,...,...,...,...,...,...,...,...,...
1925,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,7298,0,50,United-States,1
1924,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,7298,0,40,United-States,1
1923,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,5178,0,50,United-States,1
1922,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,72,United-States,1


#### 수입이 낮은 집단

In [318]:
all_case(data_dn, ['age', 'education', 'marital_status','occupation','relationship','sex',
'capital_gain','capital_loss','hours_per_week', 'native_country','num']) #workclass, race 두개 삭제

경우의 수 : 16728 

선택한 컬럼 : ['age', 'education', 'marital_status', 'occupation', 'relationship', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 18, 19] 



,age,education,marital_status,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,native_country,num
5599,29,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,19
7459,33,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,18
13623,51,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,16
9168,37,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,15
9926,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,15
...,...,...,...,...,...,...,...,...,...,...,...
6041,30,Bachelors,Never-married,Sales,Own-child,Female,0,0,65,United-States,1
6042,30,Bachelors,Never-married,Sales,Own-child,Male,0,0,18,United-States,1
6043,30,Bachelors,Never-married,Sales,Own-child,Male,0,0,40,United-States,1
6044,30,Bachelors,Never-married,Sales,Unmarried,Male,0,0,40,United-States,1


### workclass, sex 빼고

In [319]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [320]:
all_case(data_up, ['age', 'education', 'marital_status','occupation','relationship','race',
'capital_gain','capital_loss','hours_per_week', 'native_country','num']) #workclass, sex 두개 삭제

경우의 수 : 5607 

선택한 컬럼 : ['age', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 8] 



,age,education,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country,num
3489,47,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,8
2719,43,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,8
1939,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,8
3822,49,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,8
2852,44,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,0,0,40,United-States,8
...,...,...,...,...,...,...,...,...,...,...,...
1943,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,7298,0,50,United-States,1
1942,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,7298,0,40,United-States,1
1941,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,5178,0,50,United-States,1
1940,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,72,United-States,1


#### 수입이 낮은 집단

In [321]:
all_case(data_dn, ['age', 'education', 'marital_status','occupation','relationship','race',
'capital_gain','capital_loss','hours_per_week', 'native_country','num']) #workclass, sex 두개 삭제

경우의 수 : 16692 

선택한 컬럼 : ['age', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16] 



,age,education,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country,num
1153,20,Some-college,Never-married,?,Own-child,White,0,0,40,United-States,16
5458,29,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,16
7326,33,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,16
9042,37,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,15
13556,51,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,14
...,...,...,...,...,...,...,...,...,...,...,...
6035,30,HS-grad,Never-married,Machine-op-inspct,Own-child,White,0,0,44,United-States,1
6036,30,HS-grad,Never-married,Machine-op-inspct,Own-child,White,0,0,60,United-States,1
6037,30,HS-grad,Never-married,Other-service,Not-in-family,Black,0,0,40,?,1
6038,30,HS-grad,Never-married,Other-service,Not-in-family,Black,0,0,40,United-States,1


### workclass, capital_gain 빼고

In [322]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [323]:
all_case(data_up, ['age', 'education', 'marital_status','occupation','relationship','race',
'sex','capital_loss','hours_per_week', 'native_country','num']) #workclass, capital_gain 두개 삭제

경우의 수 : 5265 

선택한 컬럼 : ['age', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9] 



,age,education,marital_status,occupation,relationship,race,sex,capital_loss,hours_per_week,native_country,num
2552,43,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,9
3035,46,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,40,United-States,9
1830,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,9
3577,49,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,9
2860,45,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,40,United-States,8
...,...,...,...,...,...,...,...,...,...,...,...
1853,39,HS-grad,Married-civ-spouse,Sales,Wife,Black,Female,0,54,Dominican-Republic,1
1852,39,HS-grad,Married-civ-spouse,Sales,Husband,White,Male,1887,46,United-States,1
1850,39,HS-grad,Married-civ-spouse,Sales,Husband,White,Male,0,44,United-States,1
1849,39,HS-grad,Married-civ-spouse,Sales,Husband,White,Male,0,40,United-States,1


#### 수입이 낮은 집단

In [324]:
all_case(data_dn, ['age', 'education', 'marital_status','occupation','relationship','race',
'sex','capital_loss','hours_per_week', 'native_country','num']) #workclass, capital_gain 두개 삭제

경우의 수 : 17060 

선택한 컬럼 : ['age', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 



,age,education,marital_status,occupation,relationship,race,sex,capital_loss,hours_per_week,native_country,num
7665,33,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,17
5764,29,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,16
9400,37,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,15
10179,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,14
13935,51,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,14
...,...,...,...,...,...,...,...,...,...,...,...
6113,30,Assoc-acdm,Never-married,Prof-specialty,Unmarried,Black,Female,0,36,United-States,1
6114,30,Assoc-acdm,Never-married,Protective-serv,Not-in-family,Black,Male,0,40,United-States,1
6115,30,Assoc-acdm,Never-married,Sales,Not-in-family,White,Male,0,40,England,1
6116,30,Assoc-acdm,Never-married,Tech-support,Not-in-family,White,Female,0,40,United-States,1


### workclass, capital_loss 빼고

In [326]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [327]:
all_case(data_up, ['age', 'education', 'marital_status','occupation','relationship','race',
'sex','capital_gain','hours_per_week', 'native_country','num']) #workclass, capital_loss 두개 삭제

경우의 수 : 5444 

선택한 컬럼 : ['age', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9] 



,age,education,marital_status,occupation,relationship,race,sex,capital_gain,hours_per_week,native_country,num
2638,43,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,9
2766,44,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,40,United-States,8
3376,47,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,8
1890,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,8
3702,49,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,8
...,...,...,...,...,...,...,...,...,...,...,...
1906,39,HS-grad,Married-civ-spouse,Other-service,Husband,White,Male,0,40,United-States,1
1905,39,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,50,United-States,1
1904,39,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,45,United-States,1
1903,39,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,42,United-States,1


#### 수입이 낮은 집단

In [328]:
all_case(data_dn, ['age', 'education', 'marital_status','occupation','relationship','race',
'sex','capital_gain','hours_per_week', 'native_country','num']) #workclass, capital_loss 두개 삭제

경우의 수 : 17076 

선택한 컬럼 : ['age', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18] 



,age,education,marital_status,occupation,relationship,race,sex,capital_gain,hours_per_week,native_country,num
9409,37,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,18
5775,29,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,16
7675,33,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,16
13939,51,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,14
10183,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,14
...,...,...,...,...,...,...,...,...,...,...,...
6115,30,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,5013,45,United-States,1
6116,30,Assoc-acdm,Married-civ-spouse,Sales,Husband,White,Male,0,40,United-States,1
6117,30,Assoc-acdm,Married-civ-spouse,Sales,Husband,White,Male,0,60,Iran,1
6118,30,Assoc-acdm,Never-married,Machine-op-inspct,Not-in-family,White,Male,0,48,United-States,1


### workclass, hours_per_week 빼고

In [329]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [330]:
all_case(data_up, ['age', 'education', 'marital_status','occupation','relationship','race',
'sex','capital_gain','capital_loss', 'native_country','num']) #workclass, hours_per_week 두개 삭제

경우의 수 : 4530 

선택한 컬럼 : ['age', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14] 



,age,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,native_country,num
2434,45,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,United-States,14
1235,37,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,United-States,12
1534,39,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,United-States,12
1412,38,Bachelors,Married-civ-spouse,Sales,Husband,White,Male,0,0,United-States,12
1980,42,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,United-States,12
...,...,...,...,...,...,...,...,...,...,...,...
1644,39,Some-college,Married-civ-spouse,Other-service,Husband,Asian-Pac-Islander,Male,0,0,United-States,1
1645,39,Some-college,Married-civ-spouse,Sales,Husband,White,Male,0,0,Philippines,1
1646,39,Some-college,Married-civ-spouse,Sales,Husband,White,Male,0,0,United-States,1
1647,39,Some-college,Married-civ-spouse,Sales,Husband,White,Male,3103,0,United-States,1


#### 수입이 낮은 집단

In [331]:
all_case(data_dn, ['age', 'education', 'marital_status','occupation','relationship','race',
'sex','capital_gain','capital_loss', 'native_country','num']) #workclass, hours_per_week 두개 삭제

경우의 수 : 13872 

선택한 컬럼 : ['age', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 27, 30, 33] 



,age,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,native_country,num
750,20,Some-college,Never-married,?,Own-child,White,Female,0,0,United-States,33
548,19,Some-college,Never-married,Other-service,Own-child,White,Female,0,0,United-States,30
563,19,Some-college,Never-married,Sales,Own-child,White,Female,0,0,United-States,27
768,20,Some-college,Never-married,Adm-clerical,Own-child,White,Female,0,0,United-States,27
4209,29,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,United-States,23
...,...,...,...,...,...,...,...,...,...,...,...
5230,31,Some-college,Widowed,Adm-clerical,Unmarried,White,Female,0,0,United-States,1
5231,31,Some-college,Widowed,Exec-managerial,Unmarried,White,Female,0,0,United-States,1
5232,32,10th,Divorced,Craft-repair,Not-in-family,White,Male,0,0,United-States,1
5233,32,10th,Divorced,Machine-op-inspct,Not-in-family,White,Male,0,0,United-States,1


### workclass, native_country 빼고

In [338]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [339]:
all_case(data_up, ['age', 'education', 'marital_status','occupation','relationship','race',
'sex','capital_gain','capital_loss', 'hours_per_week','num']) #workclass, native_country 두개 삭제

경우의 수 : 5535 

선택한 컬럼 : ['age', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 8] 



,age,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,num
3010,45,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,8
1914,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,8
2817,44,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,8
3444,47,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,8
2684,43,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,8
...,...,...,...,...,...,...,...,...,...,...,...
1928,39,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,42,1
1926,39,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,38,1
1925,39,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,Asian-Pac-Islander,Male,0,0,40,1
1924,39,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,7298,0,40,1


#### 수입이 낮은 집단

In [340]:
all_case(data_dn, ['age', 'education', 'marital_status','occupation','relationship','race',
'sex','capital_gain','capital_loss', 'hours_per_week','num']) #workclass, native_country 두개 삭제

경우의 수 : 16973 

선택한 컬럼 : ['age', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17] 



,age,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,num
5714,29,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,17
9337,37,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,16
7601,33,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,16
10116,39,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,14
13855,51,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,14
...,...,...,...,...,...,...,...,...,...,...,...
6097,30,Assoc-voc,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,1
6098,30,Assoc-voc,Separated,Prof-specialty,Unmarried,White,Female,0,0,40,1
6099,30,Bachelors,Divorced,Prof-specialty,Unmarried,White,Female,0,0,40,1
6100,30,Bachelors,Married-civ-spouse,?,Wife,White,Female,0,0,4,1


### education, marital_status 빼고

In [334]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [335]:
all_case(data_up, ['age', 'workclass','occupation','relationship','race',
'sex','capital_gain','capital_loss', "hours_per_week",'native_country','num']) #education, marital_status 두개 삭제

경우의 수 : 5264 

선택한 컬럼 : ['age', 'workclass', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 



,age,workclass,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
3684,50,Private,Craft-repair,Husband,White,Male,0,0,40,United-States,11
2493,43,Private,Craft-repair,Husband,White,Male,0,0,40,United-States,10
2677,44,Private,Exec-managerial,Husband,White,Male,0,0,40,United-States,10
1396,37,Private,Craft-repair,Husband,White,Male,0,0,40,United-States,9
925,34,Private,Exec-managerial,Husband,White,Male,0,0,50,United-States,9
...,...,...,...,...,...,...,...,...,...,...,...
1854,39,Private,Sales,Husband,White,Male,0,0,40,Philippines,1
1853,39,Private,Sales,Husband,White,Male,0,0,40,Canada,1
1852,39,Private,Sales,Husband,White,Male,0,0,30,United-States,1
1851,39,Private,Prof-specialty,Wife,White,Female,7688,0,60,United-States,1


#### 수입이 낮은 집단

In [336]:
all_case(data_dn, ['age', 'workclass','occupation','relationship','race',
'sex','capital_gain','capital_loss', "hours_per_week",'native_country','num']) #education, marital_status 두개 삭제

경우의 수 : 15198 

선택한 컬럼 : ['age', 'workclass', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20] 



,age,workclass,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
8068,37,Private,Craft-repair,Husband,White,Male,0,0,40,United-States,20
8754,39,Private,Craft-repair,Husband,White,Male,0,0,40,United-States,20
7727,36,Private,Craft-repair,Husband,White,Male,0,0,40,United-States,18
427,18,Private,Sales,Own-child,White,Female,0,0,20,United-States,17
6167,32,Private,Craft-repair,Husband,White,Male,0,0,40,United-States,17
...,...,...,...,...,...,...,...,...,...,...,...
5562,30,Private,Sales,Own-child,White,Male,0,0,55,United-States,1
5563,30,Private,Sales,Unmarried,Amer-Indian-Eskimo,Female,0,0,40,United-States,1
5564,30,Private,Sales,Unmarried,Black,Female,0,0,16,United-States,1
5565,30,Private,Sales,Unmarried,Black,Female,0,0,25,United-States,1


### education, occupation 빼고

In [342]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [343]:
all_case(data_up, ['age', 'workclass','marital_status','relationship','race',
'sex','capital_gain','capital_loss', "hours_per_week",'native_country','num']) #education, occupation 두개 삭제

경우의 수 : 4233 

선택한 컬럼 : ['age', 'workclass', 'marital_status', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 22, 23, 24, 26, 27, 31, 32, 33, 38] 



,age,workclass,marital_status,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
1572,40,Private,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,38
1279,38,Private,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,33
2934,50,Private,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,32
3053,51,Private,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,31
1699,41,Private,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,31
...,...,...,...,...,...,...,...,...,...,...,...
1530,40,Federal-gov,Married-civ-spouse,Husband,Black,Male,0,0,40,United-States,1
1532,40,Federal-gov,Married-civ-spouse,Husband,White,Male,7688,0,66,United-States,1
1533,40,Federal-gov,Married-civ-spouse,Wife,White,Female,0,0,20,United-States,1
1534,40,Federal-gov,Married-civ-spouse,Wife,White,Female,0,0,65,United-States,1


#### 수입이 낮은 집단

In [344]:
all_case(data_dn, ['age', 'workclass','marital_status','relationship','race',
'sex','capital_gain','capital_loss', "hours_per_week",'native_country','num']) #education, occupation 두개 삭제

경우의 수 : 12072 

선택한 컬럼 : ['age', 'workclass', 'marital_status', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 55, 57] 



,age,workclass,marital_status,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
1116,21,Private,Never-married,Own-child,White,Male,0,0,40,United-States,57
4970,33,Private,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,55
4653,32,Private,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,55
3985,30,Private,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,52
6700,39,Private,Married-civ-spouse,Husband,White,Male,0,0,40,United-States,52
...,...,...,...,...,...,...,...,...,...,...,...
4526,31,State-gov,Married-civ-spouse,Wife,White,Female,0,0,40,United-States,1
4527,31,State-gov,Married-spouse-absent,Other-relative,White,Male,0,0,40,United-States,1
4528,31,State-gov,Never-married,Not-in-family,Asian-Pac-Islander,Male,0,0,40,United-States,1
4529,31,State-gov,Never-married,Not-in-family,White,Female,0,0,40,United-States,1


- 결혼안하고 스스로가 자식인 경우 백인 남성인 경우 수입이 낮은 집단에 속할 가능성이 매우 높다.

### education, relationship 빼고

In [342]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [345]:
all_case(data_up, ['age', 'workclass','marital_status','occupation','race',
'sex','capital_gain','capital_loss', "hours_per_week",'native_country','num']) #education, relationship 두개 삭제

경우의 수 : 5263 

선택한 컬럼 : ['age', 'workclass', 'marital_status', 'occupation', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 



,age,workclass,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
3692,50,Private,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,11
2687,44,Private,Married-civ-spouse,Exec-managerial,White,Male,0,0,40,United-States,10
2503,43,Private,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,10
1398,37,Private,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,9
932,34,Private,Married-civ-spouse,Exec-managerial,White,Male,0,0,50,United-States,9
...,...,...,...,...,...,...,...,...,...,...,...
1856,39,Private,Married-civ-spouse,Transport-moving,White,Male,15024,0,70,United-States,1
1855,39,Private,Married-civ-spouse,Transport-moving,White,Male,7298,0,48,United-States,1
1854,39,Private,Married-civ-spouse,Transport-moving,White,Male,0,1977,45,Philippines,1
1853,39,Private,Married-civ-spouse,Transport-moving,White,Male,0,0,55,United-States,1


#### 수입이 낮은 집단

In [346]:
all_case(data_dn, ['age', 'workclass','marital_status','occupation','race',
'sex','capital_gain','capital_loss', "hours_per_week",'native_country','num']) #education, relationship 두개 삭제

경우의 수 : 14769 

선택한 컬럼 : ['age', 'workclass', 'marital_status', 'occupation', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 22, 24, 25] 



,age,workclass,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
1586,22,Private,Never-married,Adm-clerical,White,Female,0,0,40,United-States,25
1998,23,Private,Never-married,Adm-clerical,White,Female,0,0,40,United-States,24
2847,25,Private,Never-married,Adm-clerical,White,Female,0,0,40,United-States,24
2033,23,Private,Never-married,Craft-repair,White,Male,0,0,40,United-States,22
8339,39,Private,Married-civ-spouse,Craft-repair,White,Male,0,0,40,United-States,20
...,...,...,...,...,...,...,...,...,...,...,...
5500,31,Private,Never-married,Sales,Black,Female,0,0,25,United-States,1
5501,31,Private,Never-married,Sales,Black,Female,0,0,40,United-States,1
5502,31,Private,Never-married,Sales,Black,Female,0,0,50,United-States,1
5503,31,Private,Never-married,Sales,Black,Female,0,0,58,United-States,1


### education, race 빼고

In [347]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [348]:
all_case(data_up, ['age', 'workclass','marital_status','occupation','relationship',
'sex','capital_gain','capital_loss', "hours_per_week",'native_country','num']) #education, race 두개 삭제

경우의 수 : 5186 

선택한 컬럼 : ['age', 'workclass', 'marital_status', 'occupation', 'relationship', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 



,age,workclass,marital_status,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,native_country,num
2647,44,Private,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,40,United-States,11
3637,50,Private,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,11
2476,43,Private,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,10
2842,45,Private,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,40,United-States,10
3499,49,Private,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,9
...,...,...,...,...,...,...,...,...,...,...,...
1829,39,Private,Married-civ-spouse,Sales,Husband,Male,0,1887,46,United-States,1
1828,39,Private,Married-civ-spouse,Sales,Husband,Male,0,1848,40,United-States,1
1827,39,Private,Married-civ-spouse,Sales,Husband,Male,0,0,60,United-States,1
1824,39,Private,Married-civ-spouse,Sales,Husband,Male,0,0,40,Philippines,1


#### 수입이 낮은 집단

In [349]:
all_case(data_dn, ['age', 'workclass','marital_status','occupation','relationship',
'sex','capital_gain','capital_loss', "hours_per_week",'native_country','num']) #education, race 두개 삭제

경우의 수 : 15146 

선택한 컬럼 : ['age', 'workclass', 'marital_status', 'occupation', 'relationship', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21] 



,age,workclass,marital_status,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,native_country,num
8693,39,Private,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,21
7995,37,Private,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,21
6440,33,Private,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,20
7643,36,Private,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,19
4780,29,Private,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,United-States,19
...,...,...,...,...,...,...,...,...,...,...,...
5576,31,Private,Divorced,Other-service,Not-in-family,Male,0,0,40,United-States,1
5577,31,Private,Divorced,Other-service,Own-child,Female,0,0,15,United-States,1
5578,31,Private,Divorced,Other-service,Own-child,Female,0,0,20,United-States,1
5579,31,Private,Divorced,Other-service,Unmarried,Female,0,0,21,United-States,1


### education, sex 빼고

In [350]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [351]:
all_case(data_up, ['age', 'workclass','marital_status','occupation','relationship',
'race','capital_gain','capital_loss', "hours_per_week",'native_country','num']) #education, sex 두개 삭제

경우의 수 : 5271 

선택한 컬럼 : ['age', 'workclass', 'marital_status', 'occupation', 'relationship', 'race', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 



,age,workclass,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country,num
3700,50,Private,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,11
2511,43,Private,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,10
2691,44,Private,Married-civ-spouse,Exec-managerial,Husband,White,0,0,40,United-States,10
928,34,Private,Married-civ-spouse,Exec-managerial,Husband,White,0,0,50,United-States,9
1234,36,Private,Married-civ-spouse,Exec-managerial,Husband,White,0,0,50,United-States,9
...,...,...,...,...,...,...,...,...,...,...,...
1855,39,Private,Married-civ-spouse,Sales,Husband,White,15024,0,50,United-States,1
1854,39,Private,Married-civ-spouse,Sales,Husband,White,7298,0,55,United-States,1
1853,39,Private,Married-civ-spouse,Sales,Husband,White,3103,0,50,United-States,1
1852,39,Private,Married-civ-spouse,Sales,Husband,White,3103,0,48,United-States,1


#### 수입이 낮은 집단

In [352]:
all_case(data_dn, ['age', 'workclass','marital_status','occupation','relationship',
'race','capital_gain','capital_loss', "hours_per_week",'native_country','num']) #education, sex 두개 삭제

경우의 수 : 15201 

선택한 컬럼 : ['age', 'workclass', 'marital_status', 'occupation', 'relationship', 'race', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 25] 



,age,workclass,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country,num
330,18,Private,Never-married,Other-service,Own-child,White,0,0,20,United-States,25
807,20,?,Never-married,?,Own-child,White,0,0,40,United-States,21
8663,39,Private,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,20
376,18,Private,Never-married,Sales,Own-child,White,0,0,20,United-States,20
7943,37,Private,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,United-States,20
...,...,...,...,...,...,...,...,...,...,...,...
5574,31,Private,Married-civ-spouse,Handlers-cleaners,Husband,White,0,0,40,Mexico,1
5576,31,Private,Married-civ-spouse,Handlers-cleaners,Husband,White,0,0,45,United-States,1
5577,31,Private,Married-civ-spouse,Handlers-cleaners,Husband,White,0,0,55,United-States,1
5578,31,Private,Married-civ-spouse,Handlers-cleaners,Husband,White,0,0,56,Mexico,1


### education, capital_gain 빼고

In [353]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [357]:
all_case(data_up, ['age', 'workclass','marital_status','occupation','relationship',
'race','sex','capital_loss', "hours_per_week",'native_country','num']) #education, capital_gain 두개 삭제

경우의 수 : 4847 

선택한 컬럼 : ['age', 'workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13] 



,age,workclass,marital_status,occupation,relationship,race,sex,capital_loss,hours_per_week,native_country,num
1494,38,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,13
3404,50,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,13
2487,44,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,40,United-States,11
2372,43,Private,Married-civ-spouse,Sales,Husband,White,Male,0,40,United-States,10
2323,43,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,10
...,...,...,...,...,...,...,...,...,...,...,...
1750,39,Self-emp-inc,Married-civ-spouse,Sales,Husband,Asian-Pac-Islander,Male,0,40,?,1
1751,39,Self-emp-inc,Married-civ-spouse,Sales,Husband,White,Male,0,48,United-States,1
1752,39,Self-emp-inc,Married-civ-spouse,Sales,Husband,White,Male,2415,60,United-States,1
1753,39,Self-emp-not-inc,Divorced,Farming-fishing,Not-in-family,White,Male,0,50,United-States,1


#### 수입이 낮은 집단

In [358]:
all_case(data_dn, ['age', 'workclass','marital_status','occupation','relationship',
'race','sex','capital_loss', "hours_per_week",'native_country','num']) #education, capital_gain 두개 삭제

경우의 수 : 15580 

선택한 컬럼 : ['age', 'workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20] 



,age,workclass,marital_status,occupation,relationship,race,sex,capital_loss,hours_per_week,native_country,num
8317,37,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,20
9039,39,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,20
5435,30,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,20
6302,32,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,19
5001,29,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,18
...,...,...,...,...,...,...,...,...,...,...,...
5704,30,Self-emp-not-inc,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,40,United-States,1
5705,30,Self-emp-not-inc,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,25,United-States,1
5706,30,Self-emp-not-inc,Married-civ-spouse,Sales,Husband,White,Male,0,50,United-States,1
5707,30,Self-emp-not-inc,Married-civ-spouse,Sales,Husband,White,Male,0,60,Germany,1


### education, capital_loss 빼고

In [362]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [363]:
all_case(data_up, ['age', 'workclass','marital_status','occupation','relationship',
'race','sex','capital_gain', "hours_per_week",'native_country','num']) #education, capital_loss 두개 삭제

경우의 수 : 5067 

선택한 컬럼 : ['age', 'workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 



,age,workclass,marital_status,occupation,relationship,race,sex,capital_gain,hours_per_week,native_country,num
3544,50,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,12
2415,43,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,12
1353,37,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,11
1195,36,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,50,United-States,10
3410,49,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,10
...,...,...,...,...,...,...,...,...,...,...,...
1807,39,Private,Never-married,Sales,Not-in-family,White,Male,0,45,United-States,1
1806,39,Private,Never-married,Prof-specialty,Not-in-family,White,Male,0,60,United-States,1
1804,39,Private,Never-married,Prof-specialty,Not-in-family,White,Male,0,40,?,1
1803,39,Private,Never-married,Prof-specialty,Not-in-family,White,Female,14084,35,United-States,1


#### 수입이 낮은 집단

In [364]:
all_case(data_dn, ['age', 'workclass','marital_status','occupation','relationship',
'race','sex','capital_gain', "hours_per_week",'native_country','num']) #education, capital_loss 두개 삭제

경우의 수 : 15630 

선택한 컬럼 : ['age', 'workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22] 



,age,workclass,marital_status,occupation,relationship,race,sex,capital_gain,hours_per_week,native_country,num
8336,37,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,22
9058,39,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,21
5453,30,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,19
6322,32,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,18
6744,33,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,United-States,18
...,...,...,...,...,...,...,...,...,...,...,...
5715,30,Self-emp-not-inc,Divorced,Craft-repair,Not-in-family,White,Male,0,49,United-States,1
5716,30,Self-emp-not-inc,Divorced,Machine-op-inspct,Not-in-family,White,Female,0,40,United-States,1
5717,30,Self-emp-not-inc,Divorced,Other-service,Unmarried,White,Female,0,30,United-States,1
5718,30,Self-emp-not-inc,Divorced,Other-service,Unmarried,White,Female,0,40,United-States,1


### education, hours_per_week 빼고

In [365]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [366]:
all_case(data_up, ['age', 'workclass','marital_status','occupation','relationship',
'race','sex','capital_gain', "capital_loss",'native_country','num']) #education, hours_per_week 두개 삭제

경우의 수 : 4102 

선택한 컬럼 : ['age', 'workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22] 



,age,workclass,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,native_country,num
2338,46,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,United-States,22
1242,38,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,United-States,21
2054,44,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,United-States,21
1516,40,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,United-States,20
729,34,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,United-States,19
...,...,...,...,...,...,...,...,...,...,...,...
1513,40,Private,Married-civ-spouse,Craft-repair,Wife,White,Female,7298,0,United-States,1
1514,40,Private,Married-civ-spouse,Exec-managerial,Husband,Asian-Pac-Islander,Male,0,0,China,1
1515,40,Private,Married-civ-spouse,Exec-managerial,Husband,Asian-Pac-Islander,Male,3103,0,India,1
1517,40,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1848,United-States,1


#### 수입이 낮은 집단

In [367]:
all_case(data_dn, ['age', 'workclass','marital_status','occupation','relationship',
'race','sex','capital_gain', "capital_loss",'native_country','num']) #education, hours_per_week 두개 삭제

경우의 수 : 11652 

선택한 컬럼 : ['age', 'workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 36, 37, 39, 41, 43, 54] 



,age,workclass,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,native_country,num
201,18,Private,Never-married,Sales,Own-child,White,Female,0,0,United-States,54
177,18,Private,Never-married,Other-service,Own-child,White,Female,0,0,United-States,43
359,19,Private,Never-married,Other-service,Own-child,White,Female,0,0,United-States,41
58,17,Private,Never-married,Other-service,Own-child,White,Male,0,0,United-States,39
389,19,Private,Never-married,Sales,Own-child,White,Female,0,0,United-States,37
...,...,...,...,...,...,...,...,...,...,...,...
4495,33,Local-gov,Never-married,Prof-specialty,Not-in-family,Black,Female,0,0,United-States,1
4496,33,Local-gov,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,Germany,1
4499,33,Local-gov,Separated,Other-service,Own-child,White,Female,0,0,United-States,1
4500,33,Local-gov,Widowed,Other-service,Unmarried,White,Female,0,0,United-States,1


### education, native_country 빼고

In [368]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [369]:
all_case(data_up, ['age', 'workclass','marital_status','occupation','relationship',
'race','sex','capital_gain', "capital_loss",'hours_per_week','num']) #education, native_country 두개 삭제

경우의 수 : 5167 

선택한 컬럼 : ['age', 'workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 



,age,workclass,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,num
3622,50,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,11
2834,45,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,10
914,34,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,10
2462,43,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,10
2641,44,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,10
...,...,...,...,...,...,...,...,...,...,...,...
1829,39,Private,Never-married,Craft-repair,Not-in-family,White,Male,0,1564,50,1
1828,39,Private,Never-married,Adm-clerical,Not-in-family,White,Female,13550,0,45,1
1827,39,Private,Married-spouse-absent,Craft-repair,Unmarried,White,Male,0,0,40,1
1826,39,Private,Married-spouse-absent,Craft-repair,Not-in-family,White,Male,0,1564,50,1


#### 수입이 낮은 집단

In [370]:
all_case(data_dn, ['age', 'workclass','marital_status','occupation','relationship',
'race','sex','capital_gain', "capital_loss",'hours_per_week','num']) #education, native_country 두개 삭제

경우의 수 : 15301 

선택한 컬럼 : ['age', 'workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20] 



,age,workclass,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,num
8830,39,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,20
7754,36,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,20
8112,37,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,20
5284,30,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,19
4871,29,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,18
...,...,...,...,...,...,...,...,...,...,...,...
5648,31,Private,Divorced,Craft-repair,Not-in-family,White,Male,0,0,38,1
5649,31,Private,Divorced,Craft-repair,Not-in-family,White,Male,0,0,40,1
5651,31,Private,Divorced,Craft-repair,Not-in-family,White,Male,0,0,80,1
5652,31,Private,Divorced,Craft-repair,Own-child,White,Male,0,0,50,1


### marital_status, occupation 빼고

In [371]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [425]:
all_case(data_up, ['age', 'workclass','education','relationship',
'race','sex','capital_gain', "capital_loss",'hours_per_week','native_country','num']) #marital_status, occupation 두개 삭제

경우의 수 : 5225 

선택한 컬럼 : ['age', 'workclass', 'education', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15] 



,age,workclass,education,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
3688,50,Private,HS-grad,Husband,White,Male,0,0,40,United-States,15
1780,39,Private,HS-grad,Husband,White,Male,0,0,40,United-States,13
2636,44,Private,Bachelors,Husband,White,Male,0,0,40,United-States,12
1574,38,Private,Bachelors,Husband,White,Male,0,0,40,United-States,11
2502,43,Private,HS-grad,Husband,White,Male,0,0,40,United-States,11
...,...,...,...,...,...,...,...,...,...,...,...
1836,39,Private,Some-college,Husband,White,Male,3103,0,50,United-States,1
1835,39,Private,Some-college,Husband,White,Male,0,2415,99,United-States,1
1834,39,Private,Some-college,Husband,White,Male,0,1977,62,United-States,1
1833,39,Private,Some-college,Husband,White,Male,0,0,52,United-States,1


#### 수입이 낮은 집단

In [424]:
all_case(data_dn, ['age', 'workclass','education','relationship','race','sex','capital_gain', 
                   "capital_loss",'hours_per_week','native_country','num']) #marital_status, occupation 두개 삭제

경우의 수 : 14527 

선택한 컬럼 : ['age', 'workclass', 'education', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28] 



,age,workclass,education,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
1350,21,Private,HS-grad,Own-child,White,Male,0,0,40,United-States,28
6237,33,Private,HS-grad,Husband,White,Male,0,0,40,United-States,28
1020,20,Private,HS-grad,Own-child,White,Male,0,0,40,United-States,26
7639,37,Private,HS-grad,Husband,White,Male,0,0,40,United-States,26
5838,32,Private,HS-grad,Husband,White,Male,0,0,40,United-States,26
...,...,...,...,...,...,...,...,...,...,...,...
5317,31,Local-gov,Some-college,Husband,White,Male,0,0,35,United-States,1
5318,31,Local-gov,Some-college,Husband,White,Male,3464,0,45,United-States,1
5319,31,Local-gov,Some-college,Not-in-family,White,Female,0,0,35,United-States,1
5320,31,Local-gov,Some-college,Own-child,Amer-Indian-Eskimo,Female,0,0,32,United-States,1


### marital_status, relationship 빼고

In [374]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [423]:
all_case(data_up, ['age', 'workclass','education','occupation','race','sex','capital_gain', 
                   "capital_loss",'hours_per_week','native_country','num']) #marital_status, relationship 두개 삭제

경우의 수 : 5808 

선택한 컬럼 : ['age', 'workclass', 'education', 'occupation', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8] 



,age,workclass,education,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
2800,43,Private,HS-grad,Craft-repair,White,Male,0,0,40,United-States,8
2946,44,Private,Bachelors,Exec-managerial,White,Male,0,0,40,United-States,7
4118,50,Private,HS-grad,Craft-repair,White,Male,0,0,40,United-States,7
3159,45,Private,Bachelors,Exec-managerial,White,Male,0,0,40,United-States,6
1973,39,Private,Bachelors,Prof-specialty,White,Male,0,0,40,United-States,6
...,...,...,...,...,...,...,...,...,...,...,...
1994,39,Private,HS-grad,Craft-repair,White,Male,7298,0,50,United-States,1
1993,39,Private,HS-grad,Craft-repair,White,Male,7298,0,40,United-States,1
1992,39,Private,HS-grad,Craft-repair,White,Male,0,2258,42,United-States,1
1991,39,Private,HS-grad,Craft-repair,White,Male,0,1564,50,United-States,1


#### 수입이 낮은 집단

In [422]:
all_case(data_dn, ['age', 'workclass','education','occupation','race','sex','capital_gain', 
                   "capital_loss",'hours_per_week','native_country','num']) #marital_status, relationship 두개 삭제

경우의 수 : 15966 

선택한 컬럼 : ['age', 'workclass', 'education', 'occupation', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20] 



,age,workclass,education,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
6545,32,Private,HS-grad,Craft-repair,White,Male,0,0,40,United-States,20
5269,29,Private,HS-grad,Craft-repair,White,Male,0,0,40,United-States,18
2961,24,Private,HS-grad,Craft-repair,White,Male,0,0,40,United-States,18
8191,36,Private,HS-grad,Craft-repair,White,Male,0,0,40,United-States,17
8545,37,Private,HS-grad,Craft-repair,White,Male,0,0,40,United-States,17
...,...,...,...,...,...,...,...,...,...,...,...
5809,30,Private,Some-college,Adm-clerical,White,Male,0,0,48,United-States,1
5811,30,Private,Some-college,Craft-repair,Black,Male,0,0,45,United-States,1
5812,30,Private,Some-college,Craft-repair,White,Male,0,0,40,?,1
5815,30,Private,Some-college,Craft-repair,White,Male,0,0,44,United-States,1


### marital_status, race 빼고

In [378]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [421]:
all_case(data_up, ['age', 'workclass','education','occupation','relationship','sex','capital_gain', 
                   "capital_loss",'hours_per_week','native_country','num']) #marital_status, race 두개 삭제

경우의 수 : 5836 

선택한 컬럼 : ['age', 'workclass', 'education', 'occupation', 'relationship', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 8] 



,age,workclass,education,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,native_country,num
2821,43,Private,HS-grad,Craft-repair,Husband,Male,0,0,40,United-States,8
2964,44,Private,Bachelors,Exec-managerial,Husband,Male,0,0,40,United-States,8
4147,50,Private,HS-grad,Craft-repair,Husband,Male,0,0,40,United-States,6
994,34,Private,Bachelors,Exec-managerial,Husband,Male,0,0,50,United-States,5
3981,49,Private,HS-grad,Craft-repair,Husband,Male,0,0,40,United-States,5
...,...,...,...,...,...,...,...,...,...,...,...
1998,39,Private,Bachelors,Tech-support,Husband,Male,0,0,50,United-States,1
1997,39,Private,Bachelors,Sales,Husband,Male,7298,0,55,United-States,1
1996,39,Private,Bachelors,Sales,Husband,Male,3103,0,48,United-States,1
1995,39,Private,Bachelors,Sales,Husband,Male,0,1848,40,United-States,1


#### 수입이 낮은 집단

In [420]:
all_case(data_dn, ['age', 'workclass','education','occupation','relationship','sex','capital_gain', 
                   "capital_loss",'hours_per_week','native_country','num']) #marital_status, race 두개 삭제

경우의 수 : 17009 

선택한 컬럼 : ['age', 'workclass', 'education', 'occupation', 'relationship', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14] 



,age,workclass,education,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,native_country,num
13844,51,Private,HS-grad,Craft-repair,Husband,Male,0,0,40,United-States,14
5718,29,Private,HS-grad,Craft-repair,Husband,Male,0,0,40,United-States,14
10059,39,Private,HS-grad,Craft-repair,Husband,Male,0,0,40,United-States,13
7573,33,Private,HS-grad,Craft-repair,Husband,Male,0,0,40,United-States,13
1053,20,?,Some-college,?,Own-child,Female,0,0,40,United-States,13
...,...,...,...,...,...,...,...,...,...,...,...
6110,30,Private,Assoc-voc,Handlers-cleaners,Husband,Male,0,0,40,Mexico,1
6111,30,Private,Assoc-voc,Handlers-cleaners,Own-child,Male,0,0,45,United-States,1
6114,30,Private,Assoc-voc,Prof-specialty,Unmarried,Female,0,0,36,United-States,1
6115,30,Private,Assoc-voc,Prof-specialty,Unmarried,Female,0,0,40,United-States,1


### marital_status, sex 빼고

In [381]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [419]:
all_case(data_up, ['age', 'workclass','education','occupation','relationship','race','capital_gain', 
                   "capital_loss",'hours_per_week','native_country','num']) #marital_status, sex 두개 삭제

경우의 수 : 5877 

선택한 컬럼 : ['age', 'workclass', 'education', 'occupation', 'relationship', 'race', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8] 



,age,workclass,education,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country,num
2841,43,Private,HS-grad,Craft-repair,Husband,White,0,0,40,United-States,8
2988,44,Private,Bachelors,Exec-managerial,Husband,White,0,0,40,United-States,7
4177,50,Private,HS-grad,Craft-repair,Husband,White,0,0,40,United-States,6
3204,45,Private,Bachelors,Exec-managerial,Husband,White,0,0,40,United-States,5
4011,49,Private,HS-grad,Craft-repair,Husband,White,0,0,40,United-States,5
...,...,...,...,...,...,...,...,...,...,...,...
2007,39,Private,Bachelors,Sales,Husband,White,7298,0,55,United-States,1
2006,39,Private,Bachelors,Sales,Husband,White,3103,0,48,United-States,1
2005,39,Private,Bachelors,Sales,Husband,White,0,1848,40,United-States,1
2004,39,Private,Bachelors,Sales,Husband,White,0,0,60,United-States,1


#### 수입이 낮은 집단

In [418]:
all_case(data_dn, ['age', 'workclass','education','occupation','relationship','race','capital_gain', 
                   "capital_loss",'hours_per_week','native_country','num']) #marital_status, sex 두개 삭제

경우의 수 : 16952 

선택한 컬럼 : ['age', 'workclass', 'education', 'occupation', 'relationship', 'race', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16] 



,age,workclass,education,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country,num
999,20,?,Some-college,?,Own-child,White,0,0,40,United-States,16
593,19,?,Some-college,?,Own-child,White,0,0,40,United-States,13
1392,21,?,Some-college,?,Own-child,White,0,0,40,United-States,12
1806,22,?,Some-college,?,Own-child,White,0,0,40,United-States,12
9945,39,Private,HS-grad,Craft-repair,Husband,White,0,0,40,United-States,12
...,...,...,...,...,...,...,...,...,...,...,...
6096,30,Private,HS-grad,Handlers-cleaners,Not-in-family,White,0,0,50,United-States,1
6097,30,Private,HS-grad,Handlers-cleaners,Other-relative,White,0,0,40,United-States,1
6098,30,Private,HS-grad,Handlers-cleaners,Own-child,Amer-Indian-Eskimo,0,0,8,United-States,1
6099,30,Private,HS-grad,Handlers-cleaners,Own-child,White,0,0,27,United-States,1


### marital_status, capital_gain 빼고

In [384]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [417]:
all_case(data_up, ['age', 'workclass','education','occupation','relationship','race','sex', 
                   "capital_loss",'hours_per_week','native_country','num']) #marital_status, capital_gain 두개 삭제

경우의 수 : 5656 

선택한 컬럼 : ['age', 'workclass', 'education', 'occupation', 'relationship', 'race', 'sex', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8] 



,age,workclass,education,occupation,relationship,race,sex,capital_loss,hours_per_week,native_country,num
4011,50,Private,HS-grad,Craft-repair,Husband,White,Male,0,40,United-States,8
2735,43,Private,HS-grad,Craft-repair,Husband,White,Male,0,40,United-States,8
2507,42,Private,Bachelors,Exec-managerial,Husband,White,Male,0,40,United-States,7
2873,44,Private,Bachelors,Exec-managerial,Husband,White,Male,0,40,United-States,7
3850,49,Private,HS-grad,Craft-repair,Husband,White,Male,0,40,United-States,6
...,...,...,...,...,...,...,...,...,...,...,...
1958,39,Private,HS-grad,Sales,Husband,White,Male,1887,46,United-States,1
1956,39,Private,HS-grad,Sales,Husband,White,Male,0,44,United-States,1
1955,39,Private,HS-grad,Sales,Husband,White,Male,0,40,United-States,1
1954,39,Private,HS-grad,Prof-specialty,Husband,White,Male,0,50,United-States,1


#### 수입이 낮은 집단

In [415]:
all_case(data_dn, ['age', 'workclass','education','occupation','relationship','race','sex', 
                   "capital_loss",'hours_per_week','native_country','num']) #marital_status, capital_gain 두개 삭제

경우의 수 : 17339 

선택한 컬럼 : ['age', 'workclass', 'education', 'occupation', 'relationship', 'race', 'sex', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 



,age,workclass,education,occupation,relationship,race,sex,capital_loss,hours_per_week,native_country,num
7779,33,Private,HS-grad,Craft-repair,Husband,White,Male,0,40,United-States,12
10314,39,Private,HS-grad,Craft-repair,Husband,White,Male,0,40,United-States,12
14157,51,Private,HS-grad,Craft-repair,Husband,White,Male,0,40,United-States,12
1090,20,?,Some-college,?,Own-child,White,Female,0,40,United-States,12
1975,22,?,Some-college,?,Own-child,White,Male,0,40,United-States,11
...,...,...,...,...,...,...,...,...,...,...,...
6180,30,Federal-gov,Bachelors,Exec-managerial,Other-relative,Asian-Pac-Islander,Female,0,45,United-States,1
6181,30,Federal-gov,HS-grad,Adm-clerical,Not-in-family,Black,Female,0,50,United-States,1
6182,30,Federal-gov,HS-grad,Adm-clerical,Other-relative,Black,Male,0,35,United-States,1
6183,30,Federal-gov,HS-grad,Armed-Forces,Other-relative,Amer-Indian-Eskimo,Male,0,48,United-States,1


### marital_status, capital_loss 빼고

In [387]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [414]:
all_case(data_up, ['age', 'workclass','education','occupation','relationship','race','sex', 
                   "capital_gain",'hours_per_week','native_country','num']) #marital_status, capital_loss 두개 삭제

경우의 수 : 5769 

선택한 컬럼 : ['age', 'workclass', 'education', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 9] 



,age,workclass,education,occupation,relationship,race,sex,capital_gain,hours_per_week,native_country,num
2779,43,Private,HS-grad,Craft-repair,Husband,White,Male,0,40,United-States,9
2922,44,Private,Bachelors,Exec-managerial,Husband,White,Male,0,40,United-States,7
1524,37,Private,Bachelors,Exec-managerial,Husband,White,Male,0,50,United-States,6
4088,50,Private,HS-grad,Craft-repair,Husband,White,Male,0,40,United-States,6
3137,45,Private,Bachelors,Exec-managerial,Husband,White,Male,0,40,United-States,5
...,...,...,...,...,...,...,...,...,...,...,...
1994,39,Private,HS-grad,Sales,Not-in-family,White,Female,4687,50,United-States,1
1993,39,Private,HS-grad,Sales,Husband,White,Male,15024,50,United-States,1
1992,39,Private,HS-grad,Sales,Husband,White,Male,3103,44,United-States,1
1991,39,Private,HS-grad,Sales,Husband,White,Male,0,50,United-States,1


#### 수입이 낮은 집단

In [413]:
all_case(data_dn, ['age', 'workclass','education','occupation','relationship','race','sex', 
                   "capital_gain",'hours_per_week','native_country','num']) #marital_status, capital_loss 두개 삭제

경우의 수 : 17350 

선택한 컬럼 : ['age', 'workclass', 'education', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 



,age,workclass,education,occupation,relationship,race,sex,capital_gain,hours_per_week,native_country,num
9514,37,Private,HS-grad,Craft-repair,Husband,White,Male,0,40,United-States,12
1084,20,?,Some-college,?,Own-child,White,Female,0,40,United-States,12
14159,51,Private,HS-grad,Craft-repair,Husband,White,Male,0,40,United-States,12
10311,39,Private,HS-grad,Craft-repair,Husband,White,Male,0,40,United-States,12
7780,33,Private,HS-grad,Craft-repair,Husband,White,Male,0,40,United-States,11
...,...,...,...,...,...,...,...,...,...,...,...
6184,30,Federal-gov,HS-grad,Armed-Forces,Other-relative,Amer-Indian-Eskimo,Male,0,48,United-States,1
6185,30,Federal-gov,Prof-school,Exec-managerial,Not-in-family,White,Male,0,55,?,1
6186,30,Federal-gov,Prof-school,Prof-specialty,Not-in-family,Black,Female,0,40,United-States,1
6187,30,Federal-gov,Some-college,Adm-clerical,Own-child,White,Male,0,40,United-States,1


### marital_status, hours_per_week 빼고

In [390]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [412]:
all_case(data_up, ['age', 'workclass','education','occupation','relationship','race','sex', 
                   "capital_gain",'capital_loss','native_country','num']) #marital_status, hours_per_week 두개 삭제

경우의 수 : 5120 

선택한 컬럼 : ['age', 'workclass', 'education', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 



,age,workclass,education,occupation,relationship,race,sex,capital_gain,capital_loss,native_country,num
2744,45,Private,Bachelors,Exec-managerial,Husband,White,Male,0,0,United-States,11
652,32,Private,Bachelors,Exec-managerial,Husband,White,Male,0,0,United-States,10
2437,43,Private,HS-grad,Craft-repair,Husband,White,Male,0,0,United-States,10
1361,37,Private,Bachelors,Exec-managerial,Husband,White,Male,0,0,United-States,10
887,34,Private,Bachelors,Exec-managerial,Husband,White,Male,0,0,United-States,10
...,...,...,...,...,...,...,...,...,...,...,...
1826,39,State-gov,Bachelors,Prof-specialty,Husband,White,Male,0,0,United-States,1
1825,39,State-gov,Bachelors,Exec-managerial,Wife,White,Female,0,0,United-States,1
1824,39,Self-emp-not-inc,Some-college,Sales,Husband,White,Male,7688,0,United-States,1
1823,39,Self-emp-not-inc,Some-college,Prof-specialty,Not-in-family,White,Male,0,1564,United-States,1


#### 수입이 낮은 집단

In [411]:
all_case(data_dn, ['age', 'workclass','education','occupation','relationship','race','sex', 
                   "capital_gain",'capital_loss','native_country','num']) #marital_status, hours_per_week 두개 삭제

경우의 수 : 14402 

선택한 컬럼 : ['age', 'workclass', 'education', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 25, 29, 33] 



,age,workclass,education,occupation,relationship,race,sex,capital_gain,capital_loss,native_country,num
647,20,?,Some-college,?,Own-child,White,Female,0,0,United-States,33
560,19,Private,Some-college,Other-service,Own-child,White,Female,0,0,United-States,29
575,19,Private,Some-college,Sales,Own-child,White,Female,0,0,United-States,25
379,19,?,Some-college,?,Own-child,White,Male,0,0,United-States,23
42,17,Private,10th,Other-service,Own-child,White,Male,0,0,United-States,21
...,...,...,...,...,...,...,...,...,...,...,...
5391,31,State-gov,Assoc-acdm,Adm-clerical,Not-in-family,White,Female,3325,0,United-States,1
5392,31,State-gov,Assoc-acdm,Adm-clerical,Own-child,Black,Female,0,0,United-States,1
5393,31,State-gov,Assoc-acdm,Other-service,Unmarried,Black,Female,0,0,United-States,1
5394,31,State-gov,Assoc-voc,Craft-repair,Husband,White,Male,0,0,United-States,1


### marital_status, native_country 빼고

In [393]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [409]:
all_case(data_up, ['age', 'workclass','education','occupation','relationship','race','sex', 
                   "capital_gain",'capital_loss','hours_per_week','num']) #marital_status, native_country 두개 삭제

경우의 수 : 5834 

선택한 컬럼 : ['age', 'workclass', 'education', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8] 



,age,workclass,education,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,num
2819,43,Private,HS-grad,Craft-repair,Husband,White,Male,0,0,40,8
2966,44,Private,Bachelors,Exec-managerial,Husband,White,Male,0,0,40,7
3180,45,Private,Bachelors,Exec-managerial,Husband,White,Male,0,0,40,7
992,34,Private,Bachelors,Exec-managerial,Husband,White,Male,0,0,50,6
4143,50,Private,HS-grad,Craft-repair,Husband,White,Male,0,0,40,6
...,...,...,...,...,...,...,...,...,...,...,...
1997,39,Private,HS-grad,Adm-clerical,Wife,White,Female,0,0,40,1
1996,39,Private,HS-grad,Adm-clerical,Husband,White,Male,0,0,40,1
1995,39,Private,Doctorate,Prof-specialty,Husband,White,Male,99999,0,55,1
1994,39,Private,Bachelors,Transport-moving,Husband,White,Male,0,1977,45,1


#### 수입이 낮은 집단

In [410]:
all_case(data_dn, ['age', 'workclass','education','occupation','relationship','race','sex', 
                   "capital_gain",'capital_loss','hours_per_week','num']) #marital_status, native_country 두개 삭제

경우의 수 : 17240 

선택한 컬럼 : ['age', 'workclass', 'education', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] 



,age,workclass,education,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,num
1078,20,?,Some-college,?,Own-child,White,Female,0,0,40,13
5829,29,Private,HS-grad,Craft-repair,Husband,White,Male,0,0,40,12
14067,51,Private,HS-grad,Craft-repair,Husband,White,Male,0,0,40,12
10237,39,Private,HS-grad,Craft-repair,Husband,White,Male,0,0,40,12
7704,33,Private,HS-grad,Craft-repair,Husband,White,Male,0,0,40,11
...,...,...,...,...,...,...,...,...,...,...,...
6164,30,Private,10th,Craft-repair,Not-in-family,White,Male,0,0,50,1
6165,30,Private,10th,Machine-op-inspct,Other-relative,Asian-Pac-Islander,Male,0,0,40,1
6166,30,Private,10th,Machine-op-inspct,Unmarried,Black,Female,0,0,40,1
6167,30,Private,10th,Other-service,Not-in-family,Amer-Indian-Eskimo,Male,0,0,40,1


### occupation, relationship 빼고

In [398]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [426]:
all_case(data_up, ['age', 'workclass','education','marital_status','race','sex', "capital_gain",'capital_loss','hours_per_week',
                   'native_country','num']) #occupation, relationship 두개 삭제

경우의 수 : 5227 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15] 



,age,workclass,education,marital_status,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
3692,50,Private,HS-grad,Married-civ-spouse,White,Male,0,0,40,United-States,15
1782,39,Private,HS-grad,Married-civ-spouse,White,Male,0,0,40,United-States,13
2640,44,Private,Bachelors,Married-civ-spouse,White,Male,0,0,40,United-States,12
1968,40,Private,HS-grad,Married-civ-spouse,White,Male,0,0,40,United-States,11
1585,38,Private,Bachelors,Married-civ-spouse,White,Male,0,0,40,United-States,11
...,...,...,...,...,...,...,...,...,...,...,...
1834,39,Private,Some-college,Married-civ-spouse,White,Female,0,0,45,United-States,1
1833,39,Private,Some-college,Married-civ-spouse,White,Female,0,0,35,Puerto-Rico,1
1832,39,Private,Some-college,Married-civ-spouse,Black,Male,0,0,52,United-States,1
1831,39,Private,Some-college,Married-civ-spouse,Black,Female,0,0,50,United-States,1


#### 수입이 낮은 집단

In [427]:
all_case(data_dn, ['age', 'workclass','education','marital_status','race','sex', "capital_gain",'capital_loss','hours_per_week',
                   'native_country','num']) #occupation, relationship 두개 삭제

경우의 수 : 14145 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 34, 39, 40, 41] 



,age,workclass,education,marital_status,race,sex,capital_gain,capital_loss,hours_per_week,native_country,num
2357,24,Private,HS-grad,Never-married,White,Male,0,0,40,United-States,41
929,20,Private,HS-grad,Never-married,White,Male,0,0,40,United-States,40
1209,21,Private,HS-grad,Never-married,White,Male,0,0,40,United-States,39
1966,23,Private,HS-grad,Never-married,White,Male,0,0,40,United-States,34
1301,21,Private,Some-college,Never-married,White,Male,0,0,40,United-States,30
...,...,...,...,...,...,...,...,...,...,...,...
5223,31,Self-emp-not-inc,Assoc-voc,Married-civ-spouse,White,Male,3137,0,50,United-States,1
5224,31,Self-emp-not-inc,Bachelors,Divorced,White,Male,0,0,60,United-States,1
5225,31,Self-emp-not-inc,Bachelors,Married-civ-spouse,White,Female,0,0,20,United-States,1
5226,31,Self-emp-not-inc,Bachelors,Married-civ-spouse,White,Male,0,0,48,United-States,1


- 백인 남성 중에 나이가 몇이냐에 따라서 나눌수도 있을것 같다.

### occupation, race 빼고

In [403]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [428]:
all_case(data_up, ['age', 'workclass','education','marital_status','relationship','sex', "capital_gain",'capital_loss','hours_per_week',
                   'native_country','num']) #occupation, race 두개 삭제

경우의 수 : 5147 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'relationship', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] 



,age,workclass,education,marital_status,relationship,sex,capital_gain,capital_loss,hours_per_week,native_country,num
3631,50,Private,HS-grad,Married-civ-spouse,Husband,Male,0,0,40,United-States,15
2596,44,Private,Bachelors,Married-civ-spouse,Husband,Male,0,0,40,United-States,14
2466,43,Private,HS-grad,Married-civ-spouse,Husband,Male,0,0,40,United-States,13
1759,39,Private,HS-grad,Married-civ-spouse,Husband,Male,0,0,40,United-States,13
1936,40,Private,HS-grad,Married-civ-spouse,Husband,Male,0,0,40,United-States,13
...,...,...,...,...,...,...,...,...,...,...,...
1815,39,Private,Some-college,Married-civ-spouse,Husband,Male,0,2415,99,United-States,1
1814,39,Private,Some-college,Married-civ-spouse,Husband,Male,0,1977,62,United-States,1
1813,39,Private,Some-college,Married-civ-spouse,Husband,Male,0,0,84,United-States,1
1810,39,Private,Some-college,Married-civ-spouse,Husband,Male,0,0,45,United-States,1


#### 수입이 낮은 집단

In [429]:
all_case(data_dn, ['age', 'workclass','education','marital_status','relationship','sex', "capital_gain",'capital_loss','hours_per_week',
                   'native_country','num']) #occupation, race 두개 삭제

경우의 수 : 14463 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'relationship', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33] 



,age,workclass,education,marital_status,relationship,sex,capital_gain,capital_loss,hours_per_week,native_country,num
6104,33,Private,HS-grad,Married-civ-spouse,Husband,Male,0,0,40,United-States,33
1289,21,Private,HS-grad,Never-married,Own-child,Male,0,0,40,United-States,32
4524,29,Private,HS-grad,Married-civ-spouse,Husband,Male,0,0,40,United-States,31
4130,28,Private,HS-grad,Married-civ-spouse,Husband,Male,0,0,40,United-States,30
1671,22,Private,HS-grad,Never-married,Own-child,Male,0,0,40,United-States,28
...,...,...,...,...,...,...,...,...,...,...,...
5303,31,Private,Doctorate,Married-spouse-absent,Not-in-family,Male,0,0,60,China,1
5304,31,Private,HS-grad,Divorced,Not-in-family,Female,0,0,25,United-States,1
5305,31,Private,HS-grad,Divorced,Not-in-family,Female,0,0,32,United-States,1
5309,31,Private,HS-grad,Divorced,Own-child,Female,0,0,15,United-States,1


### occupation, sex 빼고

In [431]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [432]:
all_case(data_up, ['age', 'workclass','education','marital_status','relationship','race', "capital_gain",'capital_loss','hours_per_week',
                   'native_country','num']) #occupation, sex 두개 삭제

경우의 수 : 5225 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'relationship', 'race', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15] 



,age,workclass,education,marital_status,relationship,race,capital_gain,capital_loss,hours_per_week,native_country,num
3688,50,Private,HS-grad,Married-civ-spouse,Husband,White,0,0,40,United-States,15
1781,39,Private,HS-grad,Married-civ-spouse,Husband,White,0,0,40,United-States,13
2638,44,Private,Bachelors,Married-civ-spouse,Husband,White,0,0,40,United-States,12
1963,40,Private,HS-grad,Married-civ-spouse,Husband,White,0,0,40,United-States,11
1578,38,Private,Bachelors,Married-civ-spouse,Husband,White,0,0,40,United-States,11
...,...,...,...,...,...,...,...,...,...,...,...
1838,39,Private,Some-college,Married-civ-spouse,Husband,White,0,2415,99,United-States,1
1837,39,Private,Some-college,Married-civ-spouse,Husband,White,0,1977,62,United-States,1
1836,39,Private,Some-college,Married-civ-spouse,Husband,White,0,0,52,United-States,1
1834,39,Private,Some-college,Married-civ-spouse,Husband,White,0,0,45,United-States,1


#### 수입이 낮은 집단

In [433]:
all_case(data_dn, ['age', 'workclass','education','marital_status','relationship','race', "capital_gain",'capital_loss','hours_per_week',
                   'native_country','num']) #occupation, sex 두개 삭제

경우의 수 : 14426 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'relationship', 'race', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 33, 35, 37, 39] 



,age,workclass,education,marital_status,relationship,race,capital_gain,capital_loss,hours_per_week,native_country,num
1279,21,Private,Some-college,Never-married,Own-child,White,0,0,40,United-States,39
1193,21,Private,HS-grad,Never-married,Own-child,White,0,0,40,United-States,37
1571,22,Private,HS-grad,Never-married,Own-child,White,0,0,40,United-States,35
899,20,Private,HS-grad,Never-married,Own-child,White,0,0,40,United-States,33
968,20,Private,Some-college,Never-married,Own-child,White,0,0,20,United-States,29
...,...,...,...,...,...,...,...,...,...,...,...
5287,31,Private,Masters,Never-married,Not-in-family,White,0,0,30,United-States,1
5288,31,Private,Masters,Never-married,Not-in-family,White,0,0,46,United-States,1
5289,31,Private,Masters,Never-married,Not-in-family,White,0,0,60,United-States,1
5290,31,Private,Preschool,Never-married,Other-relative,White,0,0,35,Mexico,1


### occupation, capital_gain 빼고

In [434]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [435]:
all_case(data_up, ['age', 'workclass','education','marital_status','relationship','race', "sex",'capital_loss','hours_per_week',
                   'native_country','num']) #occupation, capital_gain 두개 삭제

경우의 수 : 4817 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'relationship', 'race', 'sex', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 19] 



,age,workclass,education,marital_status,relationship,race,sex,capital_loss,hours_per_week,native_country,num
3396,50,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,40,United-States,19
1471,38,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,40,United-States,16
2432,44,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,40,United-States,16
1644,39,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,40,United-States,15
3533,51,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,40,United-States,13
...,...,...,...,...,...,...,...,...,...,...,...
1721,39,Self-emp-not-inc,Doctorate,Never-married,Not-in-family,White,Male,0,40,United-States,1
1722,39,Self-emp-not-inc,HS-grad,Divorced,Not-in-family,White,Male,0,50,United-States,1
1723,39,Self-emp-not-inc,HS-grad,Married-civ-spouse,Husband,White,Male,0,40,United-States,1
1724,39,Self-emp-not-inc,HS-grad,Married-civ-spouse,Husband,White,Male,0,50,United-States,1


#### 수입이 낮은 집단

In [436]:
all_case(data_dn, ['age', 'workclass','education','marital_status','relationship','race', "sex",'capital_loss','hours_per_week',
                   'native_country','num']) #occupation, capital_gain 두개 삭제

경우의 수 : 14899 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'relationship', 'race', 'sex', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 28, 29, 31] 



,age,workclass,education,marital_status,relationship,race,sex,capital_loss,hours_per_week,native_country,num
6373,33,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,40,United-States,31
5957,32,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,40,United-States,29
1345,21,Private,HS-grad,Never-married,Own-child,White,Male,0,40,United-States,28
5134,30,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,40,United-States,28
7849,37,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,40,United-States,26
...,...,...,...,...,...,...,...,...,...,...,...
5436,31,Private,12th,Never-married,Own-child,White,Male,0,32,United-States,1
5438,31,Private,1st-4th,Married-civ-spouse,Husband,White,Male,0,45,Mexico,1
5439,31,Private,1st-4th,Married-civ-spouse,Husband,White,Male,0,45,United-States,1
5440,31,Private,1st-4th,Never-married,Not-in-family,White,Male,0,37,Mexico,1


### occupation, capital_loss 빼고

In [439]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [440]:
all_case(data_up, ['age', 'workclass','education','marital_status','relationship','race', "sex",'capital_gain','hours_per_week',
                   'native_country','num']) #occupation, capital_loss 두개 삭제

경우의 수 : 5032 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'relationship', 'race', 'sex', 'capital_gain', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17] 



,age,workclass,education,marital_status,relationship,race,sex,capital_gain,hours_per_week,native_country,num
3538,50,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,40,United-States,17
2046,41,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,40,United-States,15
1711,39,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,40,United-States,14
2379,43,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,40,United-States,13
2534,44,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,40,United-States,13
...,...,...,...,...,...,...,...,...,...,...,...
1788,39,Self-emp-inc,Masters,Married-civ-spouse,Husband,White,Male,0,60,United-States,1
1787,39,Self-emp-inc,Masters,Divorced,Not-in-family,Asian-Pac-Islander,Male,0,99,Japan,1
1786,39,Self-emp-inc,HS-grad,Married-civ-spouse,Husband,White,Male,5178,50,United-States,1
1785,39,Self-emp-inc,HS-grad,Married-civ-spouse,Husband,White,Male,3103,60,United-States,1


#### 수입이 낮은 집단

In [441]:
all_case(data_dn, ['age', 'workclass','education','marital_status','relationship','race', "sex",'capital_gain','hours_per_week',
                   'native_country','num']) #occupation, capital_loss 두개 삭제

경우의 수 : 14949 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'relationship', 'race', 'sex', 'capital_gain', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28] 



,age,workclass,education,marital_status,relationship,race,sex,capital_gain,hours_per_week,native_country,num
7874,37,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,40,United-States,28
6407,33,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,40,United-States,28
1347,21,Private,HS-grad,Never-married,Own-child,White,Male,0,40,United-States,27
5987,32,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,40,United-States,27
7545,36,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,40,United-States,25
...,...,...,...,...,...,...,...,...,...,...,...
5445,31,Private,11th,Divorced,Not-in-family,White,Female,0,40,United-States,1
5446,31,Private,11th,Divorced,Other-relative,Black,Male,0,40,United-States,1
5447,31,Private,11th,Married-civ-spouse,Husband,White,Male,0,40,Mexico,1
5449,31,Private,11th,Married-civ-spouse,Husband,White,Male,0,65,Mexico,1


### occupation, hours_per_week 빼고

In [444]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [445]:
all_case(data_up, ['age', 'workclass','education','marital_status','relationship','race', "sex",'capital_gain',
                   'capital_loss','native_country','num']) #occupation, hours_per_week 두개 삭제

경우의 수 : 4098 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26] 



,age,workclass,education,marital_status,relationship,race,sex,capital_gain,capital_loss,native_country,num
1222,38,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,0,United-States,26
2158,45,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,0,United-States,24
2816,50,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,United-States,24
751,34,Private,Some-college,Married-civ-spouse,Husband,White,Male,0,0,United-States,23
1361,39,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,United-States,23
...,...,...,...,...,...,...,...,...,...,...,...
1485,40,Private,Assoc-voc,Married-civ-spouse,Husband,White,Male,7688,0,United-States,1
1486,40,Private,Bachelors,Divorced,Not-in-family,White,Female,0,0,United-States,1
1487,40,Private,Bachelors,Divorced,Not-in-family,White,Female,10520,0,United-States,1
1488,40,Private,Bachelors,Divorced,Not-in-family,White,Male,0,1564,United-States,1


#### 수입이 낮은 집단

In [446]:
all_case(data_dn, ['age', 'workclass','education','marital_status','relationship','race', "sex",'capital_gain',
                   'capital_loss','native_country','num']) #occupation, hours_per_week 두개 삭제

경우의 수 : 11170 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 32, 33, 34, 35, 36, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 52, 55, 63, 71, 73] 



,age,workclass,education,marital_status,relationship,race,sex,capital_gain,capital_loss,native_country,num
588,20,Private,Some-college,Never-married,Own-child,White,Female,0,0,United-States,73
388,19,Private,Some-college,Never-married,Own-child,White,Female,0,0,United-States,71
788,21,Private,Some-college,Never-married,Own-child,White,Female,0,0,United-States,63
594,20,Private,Some-college,Never-married,Own-child,White,Male,0,0,United-States,63
364,19,Private,HS-grad,Never-married,Own-child,White,Male,0,0,United-States,55
...,...,...,...,...,...,...,...,...,...,...,...
4207,33,Federal-gov,HS-grad,Never-married,Not-in-family,White,Female,0,0,United-States,1
4208,33,Federal-gov,HS-grad,Never-married,Own-child,White,Female,0,0,United-States,1
4209,33,Federal-gov,Masters,Married-civ-spouse,Husband,Black,Male,0,0,Nicaragua,1
4210,33,Federal-gov,Some-college,Divorced,Not-in-family,Black,Male,0,0,United-States,1


### occupation, native_country 빼고

In [447]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [448]:
all_case(data_up, ['age', 'workclass','education','marital_status','relationship','race', "sex",'capital_gain',
                   'capital_loss','hours_per_week','num']) #occupation, native_country 두개 삭제

경우의 수 : 5138 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15] 



,age,workclass,education,marital_status,relationship,race,sex,capital_gain,capital_loss,hours_per_week,num
3622,50,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,15
1749,39,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,13
2595,44,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,0,40,12
1551,38,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,0,40,12
2781,45,Private,Bachelors,Married-civ-spouse,Husband,White,Male,0,0,40,12
...,...,...,...,...,...,...,...,...,...,...,...
1812,39,Self-emp-inc,Assoc-acdm,Married-civ-spouse,Husband,White,Male,0,0,50,1
1811,39,Private,Some-college,Never-married,Not-in-family,White,Female,13550,0,45,1
1810,39,Private,Some-college,Never-married,Not-in-family,Black,Male,4787,0,40,1
1809,39,Private,Some-college,Married-spouse-absent,Unmarried,White,Male,0,0,40,1


#### 수입이 낮은 집단

In [449]:
all_case(data_dn, ['age', 'workclass','education','marital_status','relationship','race', "sex",'capital_gain',
                   'capital_loss','hours_per_week','num']) #occupation, native_country 두개 삭제

경우의 수 : 14695 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28] 



,age,workclass,education,marital_status,relationship,race,sex,capital_gain,capital_loss,hours_per_week,num
1319,21,Private,HS-grad,Never-married,Own-child,White,Male,0,0,40,28
6224,33,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,28
5815,32,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,27
7679,37,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,27
5020,30,Private,HS-grad,Married-civ-spouse,Husband,White,Male,0,0,40,26
...,...,...,...,...,...,...,...,...,...,...,...
5396,31,Private,Bachelors,Never-married,Not-in-family,White,Male,0,0,47,1
5397,31,Private,Bachelors,Never-married,Not-in-family,White,Male,0,0,48,1
5399,31,Private,Bachelors,Never-married,Not-in-family,White,Male,0,0,55,1
5400,31,Private,Bachelors,Never-married,Not-in-family,White,Male,0,1741,38,1


### relationship, race 빼고

In [450]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

#### 수입이 높은 집단

In [451]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation', "sex",'capital_gain',
                   'capital_loss','hours_per_week','native_country','num'])

경우의 수 : 5836 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 8] 



,age,workclass,education,marital_status,occupation,sex,capital_gain,capital_loss,hours_per_week,native_country,num
2823,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Male,0,0,40,United-States,8
2968,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,Male,0,0,40,United-States,8
4148,50,Private,HS-grad,Married-civ-spouse,Craft-repair,Male,0,0,40,United-States,6
2005,39,Private,HS-grad,Married-civ-spouse,Craft-repair,Male,0,0,40,United-States,5
1553,37,Private,Bachelors,Married-civ-spouse,Exec-managerial,Male,0,0,50,United-States,5
...,...,...,...,...,...,...,...,...,...,...,...
1997,39,Private,Bachelors,Never-married,Prof-specialty,Male,0,0,40,?,1
1996,39,Private,Bachelors,Never-married,Exec-managerial,Male,0,0,50,United-States,1
1995,39,Private,Bachelors,Never-married,Exec-managerial,Male,0,0,40,United-States,1
1994,39,Private,Bachelors,Married-spouse-absent,Craft-repair,Male,0,1564,50,?,1


#### 수입이 낮은 집단

In [452]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation', "sex",'capital_gain',
                   'capital_loss','hours_per_week','native_country','num'])

경우의 수 : 16598 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] 



,age,workclass,education,marital_status,occupation,sex,capital_gain,capital_loss,hours_per_week,native_country,num
2433,23,Private,HS-grad,Never-married,Adm-clerical,Female,0,0,40,United-States,15
2451,23,Private,HS-grad,Never-married,Craft-repair,Male,0,0,40,United-States,14
1736,22,?,Some-college,Never-married,?,Male,0,0,40,United-States,14
13429,51,Private,HS-grad,Married-civ-spouse,Craft-repair,Male,0,0,40,United-States,14
5325,29,Private,HS-grad,Married-civ-spouse,Craft-repair,Male,0,0,40,United-States,14
...,...,...,...,...,...,...,...,...,...,...,...
6029,31,?,HS-grad,Divorced,?,Male,0,0,75,United-States,1
6030,31,?,HS-grad,Married-civ-spouse,?,Female,0,0,30,Mexico,1
6031,31,?,HS-grad,Married-civ-spouse,?,Female,0,0,45,United-States,1
6032,31,?,HS-grad,Married-civ-spouse,?,Male,0,0,30,Mexico,1


### relationship, sex 빼고

#### 수입이 높은 집단

In [453]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation', "race",'capital_gain',
                   'capital_loss','hours_per_week','native_country','num'])

경우의 수 : 5828 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'race', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8] 



,age,workclass,education,marital_status,occupation,race,capital_gain,capital_loss,hours_per_week,native_country,num
2813,43,Private,HS-grad,Married-civ-spouse,Craft-repair,White,0,0,40,United-States,8
2960,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,White,0,0,40,United-States,7
4136,50,Private,HS-grad,Married-civ-spouse,Craft-repair,White,0,0,40,United-States,6
3177,45,Private,Bachelors,Married-civ-spouse,Exec-managerial,White,0,0,40,United-States,6
3970,49,Private,HS-grad,Married-civ-spouse,Craft-repair,White,0,0,40,United-States,6
...,...,...,...,...,...,...,...,...,...,...,...
2002,39,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,White,0,0,42,United-States,1
2000,39,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Asian-Pac-Islander,0,0,40,?,1
1999,39,Private,HS-grad,Married-civ-spouse,Exec-managerial,White,7298,0,40,United-States,1
1997,39,Private,HS-grad,Married-civ-spouse,Exec-managerial,White,0,0,50,United-States,1


#### 수입이 낮은 집단

In [454]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation', "race",'capital_gain',
                   'capital_loss','hours_per_week','native_country','num'])

경우의 수 : 16483 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'race', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16] 



,age,workclass,education,marital_status,occupation,race,capital_gain,capital_loss,hours_per_week,native_country,num
931,20,?,Some-college,Never-married,?,White,0,0,40,United-States,16
1649,22,?,Some-college,Never-married,?,White,0,0,40,United-States,15
563,19,?,Some-college,Never-married,?,White,0,0,40,United-States,15
1162,20,Private,Some-college,Never-married,Other-service,White,0,0,20,United-States,14
809,19,Private,Some-college,Never-married,Other-service,White,0,0,20,United-States,13
...,...,...,...,...,...,...,...,...,...,...,...
5983,31,Private,5th-6th,Never-married,Other-service,White,0,0,40,Mexico,1
5984,31,Private,7th-8th,Divorced,Handlers-cleaners,White,0,0,70,United-States,1
5985,31,Private,7th-8th,Divorced,Machine-op-inspct,White,0,0,40,United-States,1
5986,31,Private,7th-8th,Married-civ-spouse,Other-service,White,0,0,40,Mexico,1


### relationship, capital_gain 빼고

#### 수입이 높은 집단

In [455]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation', "race",'sex',
                   'capital_loss','hours_per_week','native_country','num'])

경우의 수 : 5655 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'race', 'sex', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8] 



,age,workclass,education,marital_status,occupation,race,sex,capital_loss,hours_per_week,native_country,num
4009,50,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,40,United-States,8
2734,43,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,40,United-States,8
2507,42,Private,Bachelors,Married-civ-spouse,Exec-managerial,White,Male,0,40,United-States,7
2874,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,White,Male,0,40,United-States,7
3085,45,Private,Bachelors,Married-civ-spouse,Exec-managerial,White,Male,0,40,United-States,6
...,...,...,...,...,...,...,...,...,...,...,...
1962,39,Private,Masters,Married-civ-spouse,Exec-managerial,White,Male,0,40,United-States,1
1961,39,Private,Masters,Married-civ-spouse,Exec-managerial,White,Female,2415,65,United-States,1
1960,39,Private,Masters,Divorced,Exec-managerial,Black,Female,0,60,United-States,1
1959,39,Private,HS-grad,Never-married,Craft-repair,White,Male,2258,42,United-States,1


#### 수입이 낮은 집단

In [456]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation', "race",'sex',
                   'capital_loss','hours_per_week','native_country','num'])

경우의 수 : 16976 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'race', 'sex', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] 



,age,workclass,education,marital_status,occupation,race,sex,capital_loss,hours_per_week,native_country,num
1819,22,?,Some-college,Never-married,?,White,Male,0,40,United-States,13
13791,51,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,40,United-States,12
1024,20,?,Some-college,Never-married,?,White,Female,0,40,United-States,12
9956,39,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,40,United-States,12
2559,23,Private,HS-grad,Never-married,Craft-repair,White,Male,0,40,United-States,12
...,...,...,...,...,...,...,...,...,...,...,...
6136,30,Private,Some-college,Married-civ-spouse,Craft-repair,White,Male,2051,40,United-States,1
6137,30,Private,Some-college,Married-civ-spouse,Exec-managerial,White,Male,0,40,United-States,1
6138,30,Private,Some-college,Married-civ-spouse,Exec-managerial,White,Male,0,50,United-States,1
6139,30,Private,Some-college,Married-civ-spouse,Farming-fishing,White,Male,0,40,United-States,1


### relationship, capital_loss 빼고

#### 수입이 높은 집단

In [457]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation', "race",'sex',
                   'capital_gain','hours_per_week','native_country','num'])

경우의 수 : 5769 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'race', 'sex', 'capital_gain', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 9] 



,age,workclass,education,marital_status,occupation,race,sex,capital_gain,hours_per_week,native_country,num
2781,43,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,40,United-States,9
2925,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,White,Male,0,40,United-States,7
1529,37,Private,Bachelors,Married-civ-spouse,Exec-managerial,White,Male,0,50,United-States,6
4089,50,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,40,United-States,6
3927,49,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,40,United-States,5
...,...,...,...,...,...,...,...,...,...,...,...
1995,39,Private,HS-grad,Married-civ-spouse,Transport-moving,White,Male,7298,48,United-States,1
1994,39,Private,HS-grad,Married-civ-spouse,Transport-moving,White,Male,0,55,United-States,1
1992,39,Private,HS-grad,Married-civ-spouse,Sales,White,Male,15024,50,United-States,1
1991,39,Private,HS-grad,Married-civ-spouse,Sales,White,Male,3103,44,United-States,1


#### 수입이 낮은 집단

In [458]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation', "race",'sex',
                   'capital_gain','hours_per_week','native_country','num'])

경우의 수 : 16991 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'race', 'sex', 'capital_gain', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] 



,age,workclass,education,marital_status,occupation,race,sex,capital_gain,hours_per_week,native_country,num
1820,22,?,Some-college,Never-married,?,White,Male,0,40,United-States,13
13800,51,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,40,United-States,12
2566,23,Private,HS-grad,Never-married,Craft-repair,White,Male,0,40,United-States,12
9963,39,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,40,United-States,12
1019,20,?,Some-college,Never-married,?,White,Female,0,40,United-States,12
...,...,...,...,...,...,...,...,...,...,...,...
6132,30,Private,Some-college,Divorced,Machine-op-inspct,White,Female,0,20,United-States,1
6133,30,Private,Some-college,Divorced,Machine-op-inspct,White,Male,0,40,United-States,1
6134,30,Private,Some-college,Divorced,Machine-op-inspct,White,Male,0,50,United-States,1
6135,30,Private,Some-college,Divorced,Other-service,White,Female,0,20,United-States,1


### relationship, hours_per_week 빼고

#### 수입이 높은 집단

In [459]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation', "race",'sex',
                   'capital_gain','capital_loss','native_country','num'])

경우의 수 : 5116 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'race', 'sex', 'capital_gain', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 



,age,workclass,education,marital_status,occupation,race,sex,capital_gain,capital_loss,native_country,num
2744,45,Private,Bachelors,Married-civ-spouse,Exec-managerial,White,Male,0,0,United-States,11
653,32,Private,Bachelors,Married-civ-spouse,Exec-managerial,White,Male,0,0,United-States,10
2432,43,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,United-States,10
3579,50,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,United-States,10
1560,38,Private,Bachelors,Married-civ-spouse,Sales,White,Male,0,0,United-States,10
...,...,...,...,...,...,...,...,...,...,...,...
1827,40,Federal-gov,Assoc-acdm,Married-civ-spouse,Prof-specialty,White,Male,0,0,United-States,1
1826,40,?,Masters,Married-civ-spouse,?,White,Male,0,0,United-States,1
1825,40,?,HS-grad,Married-civ-spouse,?,White,Female,0,0,United-States,1
1824,39,State-gov,Masters,Married-civ-spouse,Prof-specialty,White,Female,5178,0,United-States,1


#### 수입이 낮은 집단

In [460]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation', "race",'sex',
                   'capital_gain','capital_loss','native_country','num'])

경우의 수 : 13931 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'race', 'sex', 'capital_gain', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 35, 37] 



,age,workclass,education,marital_status,occupation,race,sex,capital_gain,capital_loss,native_country,num
557,20,?,Some-college,Never-married,?,White,Female,0,0,United-States,37
480,19,Private,Some-college,Never-married,Other-service,White,Female,0,0,United-States,35
490,19,Private,Some-college,Never-married,Sales,White,Female,0,0,United-States,28
720,20,Private,Some-college,Never-married,Other-service,White,Female,0,0,United-States,26
331,19,?,Some-college,Never-married,?,White,Male,0,0,United-States,25
...,...,...,...,...,...,...,...,...,...,...,...
5259,32,State-gov,Some-college,Never-married,Protective-serv,Black,Female,3887,0,United-States,1
5260,32,State-gov,Some-college,Never-married,Tech-support,Black,Female,0,0,United-States,1
5261,33,?,10th,Never-married,?,White,Male,0,0,United-States,1
5262,33,?,7th-8th,Separated,?,White,Male,0,0,Guatemala,1


### relationship, native_country 빼고

#### 수입이 높은 집단

In [461]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation', "race",'sex',
                   'capital_gain','capital_loss','hours_per_week','num'])

경우의 수 : 5834 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8] 



,age,workclass,education,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,num
2820,43,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,8
3186,45,Private,Bachelors,Married-civ-spouse,Exec-managerial,White,Male,0,0,40,7
2969,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,White,Male,0,0,40,7
996,34,Private,Bachelors,Married-civ-spouse,Exec-managerial,White,Male,0,0,50,6
4144,50,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,6
...,...,...,...,...,...,...,...,...,...,...,...
1999,39,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,7298,0,40,1
1997,39,Private,HS-grad,Married-civ-spouse,Adm-clerical,White,Male,0,0,40,1
1996,39,Private,HS-grad,Married-civ-spouse,Adm-clerical,White,Female,0,0,40,1
1995,39,Private,HS-grad,Divorced,Sales,White,Female,4687,0,50,1


#### 수입이 낮은 집단

In [462]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation', "race",'sex',
                   'capital_gain','capital_loss','hours_per_week','num'])

경우의 수 : 16844 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14] 



,age,workclass,education,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,num
2516,23,Private,HS-grad,Never-married,Craft-repair,White,Male,0,0,40,14
1780,22,?,Some-college,Never-married,?,White,Male,0,0,40,13
1005,20,?,Some-college,Never-married,?,White,Female,0,0,40,13
13666,51,Private,HS-grad,Married-civ-spouse,Craft-repair,White,Male,0,0,40,12
3029,24,Private,HS-grad,Never-married,Craft-repair,White,Male,0,0,40,12
...,...,...,...,...,...,...,...,...,...,...,...
6119,30,Self-emp-not-inc,Bachelors,Married-spouse-absent,Craft-repair,White,Male,6849,0,60,1
6120,30,Self-emp-not-inc,Bachelors,Never-married,Sales,White,Male,0,0,40,1
6121,30,Self-emp-not-inc,HS-grad,Divorced,Craft-repair,White,Male,0,0,40,1
6122,30,Self-emp-not-inc,HS-grad,Divorced,Other-service,White,Female,0,0,40,1


In [463]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

### race, sex 빼고

#### 수입이 높은 집단

In [465]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'capital_gain','capital_loss','hours_per_week','native_country','num'])

경우의 수 : 5839 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 8] 



,age,workclass,education,marital_status,occupation,relationship,capital_gain,capital_loss,hours_per_week,native_country,num
2971,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,0,0,40,United-States,8
2826,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,0,0,40,United-States,8
4151,50,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,0,0,40,United-States,6
3984,49,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,0,0,40,United-States,5
3188,45,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,0,0,40,United-States,5
...,...,...,...,...,...,...,...,...,...,...,...
1996,39,Private,Bachelors,Married-civ-spouse,Transport-moving,Husband,0,1977,45,Philippines,1
1995,39,Private,Bachelors,Married-civ-spouse,Tech-support,Husband,15024,0,40,United-States,1
1994,39,Private,Bachelors,Married-civ-spouse,Tech-support,Husband,0,0,50,United-States,1
1993,39,Private,Bachelors,Married-civ-spouse,Sales,Husband,7298,0,55,United-States,1


#### 수입이 낮은 집단

In [466]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'capital_gain','capital_loss','hours_per_week','native_country','num'])

경우의 수 : 16886 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17] 



,age,workclass,education,marital_status,occupation,relationship,capital_gain,capital_loss,hours_per_week,native_country,num
952,20,?,Some-college,Never-married,?,Own-child,0,0,40,United-States,17
13705,51,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,0,0,40,United-States,14
5449,29,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,0,0,40,United-States,14
564,19,?,Some-college,Never-married,?,Own-child,0,0,40,United-States,14
1336,21,?,Some-college,Never-married,?,Own-child,0,0,40,United-States,13
...,...,...,...,...,...,...,...,...,...,...,...
6110,30,Private,Some-college,Separated,Adm-clerical,Unmarried,0,0,40,United-States,1
6111,30,Private,Some-college,Separated,Other-service,Unmarried,0,0,40,United-States,1
6112,30,Private,Some-college,Separated,Sales,Unmarried,0,0,16,United-States,1
6113,30,Private,Some-college,Separated,Transport-moving,Not-in-family,0,0,20,United-States,1


### race, capital_gain 빼고

#### 수입이 높은 집단

In [467]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'sex','capital_loss','hours_per_week','native_country','num'])

경우의 수 : 5599 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'sex', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8] 



,age,workclass,education,marital_status,occupation,relationship,sex,capital_loss,hours_per_week,native_country,num
3968,50,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,40,United-States,8
2709,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,40,United-States,8
2845,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Male,0,40,United-States,8
2485,42,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Male,0,40,United-States,7
1923,39,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,40,United-States,6
...,...,...,...,...,...,...,...,...,...,...,...
1947,39,Private,Masters,Divorced,Exec-managerial,Unmarried,Female,0,60,United-States,1
1946,39,Private,HS-grad,Never-married,Craft-repair,Own-child,Male,2258,42,United-States,1
1945,39,Private,HS-grad,Never-married,Craft-repair,Not-in-family,Male,1564,50,United-States,1
1944,39,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,Male,0,55,United-States,1


#### 수입이 낮은 집단

In [468]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'sex','capital_loss','hours_per_week','native_country','num'])

경우의 수 : 17269 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'sex', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14] 



,age,workclass,education,marital_status,occupation,relationship,sex,capital_loss,hours_per_week,native_country,num
7683,33,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,40,United-States,14
14094,51,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,40,United-States,14
5774,29,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,40,United-States,14
10238,39,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,40,United-States,13
1053,20,?,Some-college,Never-married,?,Own-child,Female,0,40,United-States,13
...,...,...,...,...,...,...,...,...,...,...,...
6189,30,Private,Bachelors,Never-married,Adm-clerical,Not-in-family,Male,0,50,United-States,1
6190,30,Private,Bachelors,Never-married,Adm-clerical,Other-relative,Male,0,40,United-States,1
6191,30,Private,Bachelors,Never-married,Craft-repair,Own-child,Male,0,10,United-States,1
6192,30,Private,Bachelors,Never-married,Exec-managerial,Not-in-family,Female,0,40,United-States,1


### race, capital_loss 빼고

#### 수입이 높은 집단

In [469]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'sex','capital_gain','hours_per_week','native_country','num'])

경우의 수 : 5721 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'sex', 'capital_gain', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 9] 



,age,workclass,education,marital_status,occupation,relationship,sex,capital_gain,hours_per_week,native_country,num
2903,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Male,0,40,United-States,9
2761,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,40,United-States,9
2357,41,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,40,United-States,6
1518,37,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Male,0,50,United-States,6
4054,50,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,40,United-States,6
...,...,...,...,...,...,...,...,...,...,...,...
1978,39,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,Male,0,45,?,1
1977,39,Private,HS-grad,Married-civ-spouse,Other-service,Husband,Male,0,60,United-States,1
1976,39,Private,HS-grad,Married-civ-spouse,Other-service,Husband,Male,0,40,United-States,1
1975,39,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,Male,0,50,United-States,1


#### 수입이 낮은 집단

In [470]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'sex','capital_gain','hours_per_week','native_country','num'])

경우의 수 : 17276 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'sex', 'capital_gain', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14] 



,age,workclass,education,marital_status,occupation,relationship,sex,capital_gain,hours_per_week,native_country,num
5774,29,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,40,United-States,14
14094,51,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,40,United-States,14
1046,20,?,Some-college,Never-married,?,Own-child,Female,0,40,United-States,13
10231,39,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,40,United-States,13
7685,33,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,40,United-States,13
...,...,...,...,...,...,...,...,...,...,...,...
6194,30,Private,Bachelors,Never-married,Exec-managerial,Not-in-family,Male,0,44,United-States,1
6195,30,Private,Bachelors,Never-married,Exec-managerial,Own-child,Female,0,52,Taiwan,1
6196,30,Private,Bachelors,Never-married,Exec-managerial,Own-child,Male,0,40,Germany,1
6197,30,Private,Bachelors,Never-married,Machine-op-inspct,Not-in-family,Female,0,40,United-States,1


### race, hours_per_week 빼고

#### 수입이 높은 집단

In [471]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'sex','capital_gain','capital_loss','native_country','num'])

경우의 수 : 5040 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'sex', 'capital_gain', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 



,age,workclass,education,marital_status,occupation,relationship,sex,capital_gain,capital_loss,native_country,num
2703,45,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,United-States,11
2523,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,United-States,11
882,34,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,United-States,11
1345,37,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,United-States,10
3518,50,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,United-States,10
...,...,...,...,...,...,...,...,...,...,...,...
1805,40,?,HS-grad,Married-civ-spouse,?,Wife,Female,0,0,United-States,1
1804,39,State-gov,Masters,Married-civ-spouse,Prof-specialty,Wife,Female,5178,0,United-States,1
1803,39,State-gov,Doctorate,Married-civ-spouse,Prof-specialty,Husband,Male,0,0,United-States,1
1802,39,State-gov,Doctorate,Divorced,Prof-specialty,Not-in-family,Female,0,0,United-States,1


#### 수입이 낮은 집단

In [472]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'sex','capital_gain','capital_loss','native_country','num'])

경우의 수 : 14407 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'sex', 'capital_gain', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 28, 35] 



,age,workclass,education,marital_status,occupation,relationship,sex,capital_gain,capital_loss,native_country,num
587,20,?,Some-college,Never-married,?,Own-child,Female,0,0,United-States,35
510,19,Private,Some-college,Never-married,Other-service,Own-child,Female,0,0,United-States,28
523,19,Private,Some-college,Never-married,Sales,Own-child,Female,0,0,United-States,25
1066,21,Private,Some-college,Never-married,Sales,Own-child,Female,0,0,United-States,24
740,20,Private,Some-college,Never-married,Adm-clerical,Own-child,Female,0,0,United-States,24
...,...,...,...,...,...,...,...,...,...,...,...
5414,32,Private,Bachelors,Never-married,Sales,Not-in-family,Female,0,1590,United-States,1
5415,32,Private,Bachelors,Never-married,Sales,Not-in-family,Male,0,0,United-States,1
5416,32,Private,Bachelors,Never-married,Tech-support,Not-in-family,Female,0,0,Philippines,1
5417,32,Private,Bachelors,Never-married,Tech-support,Not-in-family,Male,0,0,United-States,1


### race, native_country 빼고

#### 수입이 높은 집단

In [473]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'sex','capital_gain','capital_loss','hours_per_week','num'])

경우의 수 : 5764 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8] 



,age,workclass,education,marital_status,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,num
2783,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,8
2928,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,40,8
3141,45,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,40,7
988,34,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,50,6
4086,50,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,6
...,...,...,...,...,...,...,...,...,...,...,...
1982,39,Private,HS-grad,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,40,1
1981,39,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,7298,0,50,1
1980,39,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,7298,0,40,1
1978,39,Private,HS-grad,Married-civ-spouse,Adm-clerical,Wife,Female,0,0,40,1


#### 수입이 낮은 집단

In [474]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'sex','capital_gain','capital_loss','hours_per_week','num'])

경우의 수 : 17078 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15] 



,age,workclass,education,marital_status,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,num
5679,29,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,15
13914,51,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,14
1037,20,?,Some-college,Never-married,?,Own-child,Female,0,0,40,14
10089,39,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Male,0,0,40,13
1872,22,?,Some-college,Never-married,?,Own-child,Male,0,0,40,13
...,...,...,...,...,...,...,...,...,...,...,...
6159,30,Private,HS-grad,Married-civ-spouse,Handlers-cleaners,Husband,Male,0,0,60,1
6160,30,Private,HS-grad,Married-civ-spouse,Handlers-cleaners,Husband,Male,0,0,66,1
6162,30,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,Male,0,0,45,1
6163,30,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,Male,5013,0,40,1


In [475]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

### sex, capital_gain 빼고

#### 수입이 높은 집단

In [476]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','capital_loss','hours_per_week','native_country','num'])

경우의 수 : 5658 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8] 



,age,workclass,education,marital_status,occupation,relationship,race,capital_loss,hours_per_week,native_country,num
2739,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,40,United-States,8
4013,50,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,40,United-States,8
2879,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,0,40,United-States,7
2511,42,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,0,40,United-States,7
3088,45,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,0,40,United-States,6
...,...,...,...,...,...,...,...,...,...,...,...
1957,39,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,0,44,United-States,1
1956,39,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,0,40,United-States,1
1955,39,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,0,50,United-States,1
1954,39,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,0,45,?,1


#### 수입이 낮은 집단

In [477]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','capital_loss','hours_per_week','native_country','num'])

경우의 수 : 17193 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'capital_loss', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16] 



,age,workclass,education,marital_status,occupation,relationship,race,capital_loss,hours_per_week,native_country,num
1000,20,?,Some-college,Never-married,?,Own-child,White,0,40,United-States,16
1396,21,?,Some-college,Never-married,?,Own-child,White,0,40,United-States,13
593,19,?,Some-college,Never-married,?,Own-child,White,0,40,United-States,13
1810,22,?,Some-college,Never-married,?,Own-child,White,0,40,United-States,12
7546,33,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,40,United-States,12
...,...,...,...,...,...,...,...,...,...,...,...
6174,30,Private,HS-grad,Never-married,Handlers-cleaners,Other-relative,White,0,40,United-States,1
6175,30,Private,HS-grad,Never-married,Handlers-cleaners,Own-child,Amer-Indian-Eskimo,0,8,United-States,1
6176,30,Private,HS-grad,Never-married,Handlers-cleaners,Own-child,White,0,27,United-States,1
6178,30,Private,HS-grad,Never-married,Handlers-cleaners,Unmarried,Black,0,72,United-States,1


### sex, capital_loss 빼고

#### 수입이 높은 집단

In [478]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','capital_gain','hours_per_week','native_country','num'])

경우의 수 : 5771 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'capital_gain', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 9] 



,age,workclass,education,marital_status,occupation,relationship,race,capital_gain,hours_per_week,native_country,num
2783,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,40,United-States,9
2928,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,0,40,United-States,7
1527,37,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,0,50,United-States,6
4091,50,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,40,United-States,6
3579,47,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,40,United-States,5
...,...,...,...,...,...,...,...,...,...,...,...
1993,39,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,0,50,United-States,1
1992,39,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,0,46,United-States,1
1991,39,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,0,40,United-States,1
1990,39,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,0,50,United-States,1


#### 수입이 낮은 집단

In [479]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','capital_gain','hours_per_week','native_country','num'])

경우의 수 : 17199 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'capital_gain', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16] 



,age,workclass,education,marital_status,occupation,relationship,race,capital_gain,hours_per_week,native_country,num
991,20,?,Some-college,Never-married,?,Own-child,White,0,40,United-States,16
587,19,?,Some-college,Never-married,?,Own-child,White,0,40,United-States,14
1388,21,?,Some-college,Never-married,?,Own-child,White,0,40,United-States,12
13997,51,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,40,United-States,12
10107,39,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,40,United-States,12
...,...,...,...,...,...,...,...,...,...,...,...
6181,30,Private,HS-grad,Never-married,Machine-op-inspct,Other-relative,White,0,35,Puerto-Rico,1
6182,30,Private,HS-grad,Never-married,Machine-op-inspct,Other-relative,White,0,40,Vietnam,1
6183,30,Private,HS-grad,Never-married,Machine-op-inspct,Own-child,White,0,40,United-States,1
6184,30,Private,HS-grad,Never-married,Machine-op-inspct,Own-child,White,0,44,United-States,1


### sex, hours_per_week 빼고

#### 수입이 높은 집단

In [480]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','capital_gain','capital_loss','native_country','num'])

경우의 수 : 5121 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'capital_gain', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 



,age,workclass,education,marital_status,occupation,relationship,race,capital_gain,capital_loss,native_country,num
2749,45,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,0,0,United-States,11
1361,37,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,0,0,United-States,10
890,34,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,0,0,United-States,10
2438,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,United-States,10
653,32,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,0,0,United-States,10
...,...,...,...,...,...,...,...,...,...,...,...
1829,39,State-gov,Masters,Married-civ-spouse,Prof-specialty,Wife,White,5178,0,United-States,1
1828,39,State-gov,Doctorate,Married-civ-spouse,Prof-specialty,Husband,White,0,0,United-States,1
1827,39,State-gov,Doctorate,Divorced,Prof-specialty,Not-in-family,White,0,0,United-States,1
1826,39,State-gov,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,0,0,United-States,1


#### 수입이 낮은 집단

In [481]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','capital_gain','capital_loss','native_country','num'])

경우의 수 : 14455 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'capital_gain', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 26, 30, 31, 32, 36, 41, 50] 



,age,workclass,education,marital_status,occupation,relationship,race,capital_gain,capital_loss,native_country,num
569,20,?,Some-college,Never-married,?,Own-child,White,0,0,United-States,50
325,19,?,Some-college,Never-married,?,Own-child,White,0,0,United-States,41
493,19,Private,Some-college,Never-married,Other-service,Own-child,White,0,0,United-States,41
751,20,Private,Some-college,Never-married,Other-service,Own-child,White,0,0,United-States,36
503,19,Private,Some-college,Never-married,Sales,Own-child,White,0,0,United-States,36
...,...,...,...,...,...,...,...,...,...,...,...
5398,32,Private,HS-grad,Married-civ-spouse,Sales,Husband,Asian-Pac-Islander,0,0,Hong,1
5400,32,Private,HS-grad,Married-civ-spouse,Sales,Wife,White,0,0,United-States,1
5401,32,Private,HS-grad,Married-civ-spouse,Tech-support,Wife,White,0,0,United-States,1
5403,32,Private,HS-grad,Married-civ-spouse,Transport-moving,Own-child,White,0,0,Mexico,1


### sex, native_country 빼고

#### 수입이 높은 집단

In [482]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','capital_gain','capital_loss','hours_per_week','num'])

경우의 수 : 5836 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'capital_gain', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8] 



,age,workclass,education,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,num
2823,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,8
3188,45,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,0,0,40,7
2972,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,0,0,40,7
4146,50,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,6
997,34,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,0,0,50,6
...,...,...,...,...,...,...,...,...,...,...,...
1997,39,Private,Doctorate,Married-civ-spouse,Prof-specialty,Husband,White,99999,0,55,1
1996,39,Private,Bachelors,Widowed,Prof-specialty,Unmarried,White,0,0,50,1
1995,39,Private,Bachelors,Separated,Prof-specialty,Unmarried,Asian-Pac-Islander,0,0,40,1
1993,39,Private,Bachelors,Never-married,Exec-managerial,Not-in-family,White,0,0,50,1


#### 수입이 낮은 집단

In [483]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','capital_gain','capital_loss','hours_per_week','num'])

경우의 수 : 17099 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'capital_gain', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 18] 



,age,workclass,education,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,num
984,20,?,Some-college,Never-married,?,Own-child,White,0,0,40,18
584,19,?,Some-college,Never-married,?,Own-child,White,0,0,40,13
1773,22,?,Some-college,Never-married,?,Own-child,White,0,0,40,13
5562,29,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,12
13908,51,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,0,0,40,12
...,...,...,...,...,...,...,...,...,...,...,...
6169,30,Private,Some-college,Divorced,Sales,Unmarried,Amer-Indian-Eskimo,0,0,40,1
6170,30,Private,Some-college,Divorced,Tech-support,Own-child,White,0,0,40,1
6171,30,Private,Some-college,Divorced,Transport-moving,Not-in-family,White,0,0,40,1
6172,30,Private,Some-college,Married-civ-spouse,Adm-clerical,Husband,White,0,0,30,1


In [484]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

### capital_gain, capital_loss 빼고

#### 수입이 높은 집단

In [485]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','sex','hours_per_week','native_country','num'])

경우의 수 : 5533 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,num
2668,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,9
3912,50,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,8
2449,42,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,40,United-States,7
2804,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,40,United-States,7
1892,39,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,6
...,...,...,...,...,...,...,...,...,...,...,...
1937,39,Private,Prof-school,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,1
1936,39,Private,Prof-school,Divorced,Prof-specialty,Not-in-family,White,Male,70,United-States,1
1935,39,Private,Masters,Never-married,Sales,Not-in-family,White,Male,50,United-States,1
1934,39,Private,Masters,Never-married,Sales,Not-in-family,White,Male,45,United-States,1


#### 수입이 낮은 집단

In [486]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','sex','hours_per_week','native_country','num'])

경우의 수 : 17542 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'hours_per_week', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,num
14347,51,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,12
7376,32,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,12
1083,20,?,Some-college,Never-married,?,Own-child,White,Female,40,United-States,12
9649,37,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,12
7865,33,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,12
...,...,...,...,...,...,...,...,...,...,...,...
6253,30,Local-gov,Some-college,Never-married,Transport-moving,Unmarried,Black,Female,40,United-States,1
6254,30,Private,10th,Divorced,Other-service,Not-in-family,Amer-Indian-Eskimo,Male,40,United-States,1
6256,30,Private,10th,Married-civ-spouse,Transport-moving,Husband,White,Male,55,United-States,1
6257,30,Private,10th,Never-married,Craft-repair,Not-in-family,White,Male,50,United-States,1


### capital_gain, hours_per_week 빼고

#### 수입이 높은 집단

In [487]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','sex','capital_loss','native_country','num'])

경우의 수 : 4647 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_loss,native_country,num
2638,46,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,United-States,15
1247,37,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,United-States,14
1414,38,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,United-States,14
617,32,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,United-States,14
3225,50,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,United-States,13
...,...,...,...,...,...,...,...,...,...,...,...
1699,40,Private,Assoc-voc,Married-civ-spouse,Tech-support,Husband,White,Male,0,United-States,1
1701,40,Private,Bachelors,Divorced,Sales,Not-in-family,White,Male,1564,United-States,1
1703,40,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Asian-Pac-Islander,Male,0,China,1
1705,40,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,1902,United-States,1


#### 수입이 낮은 집단

In [488]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','sex','capital_loss','native_country','num'])

경우의 수 : 14852 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_loss', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 28, 33] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_loss,native_country,num
644,20,?,Some-college,Never-married,?,Own-child,White,Female,0,United-States,33
556,19,Private,Some-college,Never-married,Other-service,Own-child,White,Female,0,United-States,28
571,19,Private,Some-college,Never-married,Sales,Own-child,White,Female,0,United-States,25
373,19,?,Some-college,Never-married,?,Own-child,White,Male,0,United-States,23
810,20,Private,Some-college,Never-married,Adm-clerical,Own-child,White,Female,0,United-States,22
...,...,...,...,...,...,...,...,...,...,...,...
5530,31,Self-emp-not-inc,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,United-States,1
5531,31,Self-emp-not-inc,Some-college,Married-civ-spouse,Other-service,Wife,White,Female,0,United-States,1
5532,31,State-gov,12th,Married-civ-spouse,Other-service,Wife,White,Female,0,United-States,1
5533,31,State-gov,Assoc-acdm,Never-married,Adm-clerical,Not-in-family,White,Female,0,United-States,1


### capital_gain, native_country 빼고

#### 수입이 높은 집단

In [489]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','sex','capital_loss','hours_per_week','num'])

경우의 수 : 5597 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_loss,hours_per_week,num
3969,50,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,8
3057,45,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,40,8
2712,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,8
2852,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,40,7
2486,42,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,40,7
...,...,...,...,...,...,...,...,...,...,...,...
1942,39,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,White,Male,0,48,1
1940,39,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,Male,1887,46,1
1938,39,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,Male,0,44,1
1937,39,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,Male,0,40,1


#### 수입이 낮은 집단

In [490]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','sex','capital_loss','hours_per_week','num'])

경우의 수 : 17451 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_loss', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_loss,hours_per_week,num
1077,20,?,Some-college,Never-married,?,Own-child,White,Female,0,40,13
10390,39,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,12
5875,29,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,12
7317,32,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,12
7803,33,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,12
...,...,...,...,...,...,...,...,...,...,...,...
6233,30,Private,7th-8th,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,1
6234,30,Private,7th-8th,Married-civ-spouse,Craft-repair,Husband,White,Male,0,48,1
6235,30,Private,7th-8th,Never-married,Craft-repair,Not-in-family,White,Male,0,50,1
6236,30,Private,7th-8th,Never-married,Machine-op-inspct,Unmarried,White,Male,0,42,1


In [491]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

### capital_loss, hours_per_week 빼고

#### 수입이 높은 집단

In [492]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','sex','capital_gain','native_country','num'])

경우의 수 : 4876 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,native_country,num
2601,45,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,United-States,13
2305,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,United-States,12
1298,37,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,United-States,12
856,34,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,United-States,12
632,32,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,United-States,12
...,...,...,...,...,...,...,...,...,...,...,...
1768,40,Private,12th,Married-civ-spouse,Craft-repair,Husband,White,Male,0,United-States,1
1769,40,Private,7th-8th,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,Mexico,1
1770,40,Private,Assoc-acdm,Divorced,Exec-managerial,Not-in-family,White,Male,0,United-States,1
1772,40,Private,Assoc-acdm,Married-civ-spouse,Prof-specialty,Husband,White,Male,7298,United-States,1


#### 수입이 낮은 집단

In [493]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','sex','capital_gain','native_country','num'])

경우의 수 : 14898 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'native_country', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 23, 26, 28, 33] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,native_country,num
639,20,?,Some-college,Never-married,?,Own-child,White,Female,0,United-States,33
554,19,Private,Some-college,Never-married,Other-service,Own-child,White,Female,0,United-States,28
568,19,Private,Some-college,Never-married,Sales,Own-child,White,Female,0,United-States,26
370,19,?,Some-college,Never-married,?,Own-child,White,Male,0,United-States,23
841,20,Private,Some-college,Never-married,Other-service,Own-child,White,Male,0,United-States,21
...,...,...,...,...,...,...,...,...,...,...,...
5538,31,Self-emp-not-inc,HS-grad,Never-married,Other-service,Unmarried,White,Female,0,United-States,1
5539,31,Self-emp-not-inc,Masters,Separated,Tech-support,Not-in-family,White,Female,0,United-States,1
5541,31,Self-emp-not-inc,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,United-States,1
5542,31,Self-emp-not-inc,Some-college,Married-civ-spouse,Other-service,Wife,White,Female,0,United-States,1


### capital_loss, native_country 빼고

#### 수입이 높은 집단

In [494]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','sex','capital_gain','hours_per_week','num'])

경우의 수 : 5715 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 9] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,hours_per_week,num
2756,43,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,9
3115,45,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,40,7
2901,44,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,40,7
983,34,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,50,6
1404,36,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,6
...,...,...,...,...,...,...,...,...,...,...,...
1976,39,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,Male,3103,44,1
1975,39,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,Male,0,50,1
1974,39,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,Male,0,46,1
1973,39,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,Male,0,40,1


#### 수입이 낮은 집단

In [495]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','sex','capital_gain','hours_per_week','num'])

경우의 수 : 17469 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'hours_per_week', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,hours_per_week,num
1073,20,?,Some-college,Never-married,?,Own-child,White,Female,0,40,13
5881,29,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,12
9593,37,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,12
10393,39,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,12
14279,51,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,40,12
...,...,...,...,...,...,...,...,...,...,...,...
6238,30,Private,7th-8th,Divorced,Transport-moving,Not-in-family,White,Male,0,40,1
6239,30,Private,7th-8th,Married-civ-spouse,Craft-repair,Husband,White,Male,0,48,1
6240,30,Private,7th-8th,Married-civ-spouse,Craft-repair,Husband,White,Male,2829,40,1
6241,30,Private,7th-8th,Never-married,Craft-repair,Not-in-family,White,Male,0,50,1


In [496]:
train.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'income', 'num'],
      dtype='object')

### hours_per_week, native_country 빼고

#### 수입이 높은 집단

In [497]:
all_case(data_up, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','sex','capital_gain','capital_loss','num'])

경우의 수 : 5033 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 14] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,num
2699,45,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,14
877,34,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,11
1341,37,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,10
761,33,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,10
642,32,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...
1800,40,Federal-gov,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,7688,0,1
1799,40,Federal-gov,Assoc-voc,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,0,1
1798,40,Federal-gov,Assoc-acdm,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,1
1797,40,?,Masters,Married-civ-spouse,?,Husband,White,Male,0,0,1


#### 수입이 낮은 집단

In [498]:
all_case(data_dn, ['age', 'workclass','education','marital_status','occupation','relationship',
                   'race','sex','capital_gain','capital_loss','num'])

경우의 수 : 14742 

선택한 컬럼 : ['age', 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'num'] 

경우 중 숫자의 개수 unique : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 25, 28, 36] 



,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,num
628,20,?,Some-college,Never-married,?,Own-child,White,Female,0,0,36
544,19,Private,Some-college,Never-married,Other-service,Own-child,White,Female,0,0,28
786,20,Private,Some-college,Never-married,Adm-clerical,Own-child,White,Female,0,0,25
367,19,?,Some-college,Never-married,?,Own-child,White,Male,0,0,25
558,19,Private,Some-college,Never-married,Sales,Own-child,White,Female,0,0,25
...,...,...,...,...,...,...,...,...,...,...,...
5520,32,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,1
5521,32,Private,11th,Never-married,Craft-repair,Not-in-family,Asian-Pac-Islander,Male,0,0,1
5522,32,Private,11th,Never-married,Other-service,Unmarried,Black,Female,0,0,1
5523,32,Private,11th,Never-married,Priv-house-serv,Unmarried,Black,Female,0,0,1
